In [1]:
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error, mean_absolute_error
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor

# Import dataset

In [2]:
data_jaktim = pd.read_csv('data/data_house_price_jaktim.csv', encoding = "ISO-8859-1")
data_jakpus = pd.read_csv('data/data_house_price_jakpus.csv', encoding = "ISO-8859-1")
data_jaksel = pd.read_csv('data/data_house_price_jaksel.csv', encoding = "ISO-8859-1")
data_jakbar = pd.read_csv('data/data_house_price_jakbar.csv', encoding = "ISO-8859-1")
data_jakut = pd.read_csv('data/data_house_price_jakut.csv', encoding = "ISO-8859-1")

In [3]:
data_jaktim

,source,title,date of collecting,region,seller's name,latest modified date,price,Kamar Tidur,Kamar Mandi,Luas Tanah,...,Perbelanjaan,Transportasi,Kesehatan,Kamar Pembantu,Kamar Mandi Pembantu,Tahun Di Renovasi,Wisata,Others,Sekolah,Ibadah
0,https://www.rumah123.com/properti/jakarta-timu...,Rumah Dijual msh Gress Dekat Aeon Mall Dan...,1/25/2024 14:23,"Cakung, Jakarta Timur",Lastry,04 Januari 2024,"Rp 2,2 Miliar",3.0,2.0,84 m²,...,NaN,"Stasiun Cakung, Stasiun Buaran, Stasiun Klende...",NaN,NaN,NaN,NaN,NaN,Walikota Jakarta Timur,NaN,NaN
1,https://www.rumah123.com/properti/jakarta-timu...,RUMAH EXCLUSIVE 2 LANTAI LOKASI STRATEGIS DEKA...,1/25/2024 14:24,"Duren Sawit, Jakarta Timur",Erni Rahmawati,30 Desember 2023,"Rp 1,7 Miliar",3.0,2.0,118 m²,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Duren Sawit, Pondok Kopi Bekasi",NaN,NaN
2,https://www.rumah123.com/properti/jakarta-timu...,Rumah semi kantor Asri di Jl. Cipinang Baru Ra...,1/25/2024 14:24,"Cipinang, Jakarta Timur",Wawan,25 Januari 2024,"Rp 2,47 Miliar",2.0,2.0,76 m²,...,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN
3,https://www.rumah123.com/properti/jakarta-timu...,"Mau beli rumah? Ada loh dipisangan Jaktim,rmh ...",1/25/2024 14:24,"Matraman, Jakarta Timur",Debora,08 Januari 2024,Rp 828 Juta,2.0,2.0,31 m²,...,Pasar Jangkrik,"Stasiun Jatinegara, Stasiun Pondok Jati",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://www.rumah123.com/properti/jakarta-timu...,Mau punya rumah di lok.strategis? Ada loh di j...,1/25/2024 14:24,"Pulo Gadung, Jakarta Timur",Debora,12 Januari 2024,"Rp 1,32 Miliar",3.0,2.0,48 m²,...,NaN,"Stasiun Klender, Stasiun Cipinang, Terminal Ra...",RS Persahabatan,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5449,https://www.rumah123.com/properti/jakarta-timu...,Rumah kost 67 Kamar di Rawamangun Full Terisi,2/18/2024 20:36,"Rawamangun, Jakarta Timur",Baim Wong,18 Februari 2024,Rp 11 Miliar,67.0,67.0,404 m²,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5450,https://www.rumah123.com/properti/jakarta-timu...,Rumah 2lt 7x15 2+1kt Cluster Mahakam Jgc Jakar...,2/18/2024 20:36,"Cakung, Jakarta Timur",Stephanie Chandra,18 Februari 2024,"Rp 1,95 Miliar",2.0,3.0,105 m²,...,NaN,"Stasiun Cakung, Stasiun Buaran, Stasiun Klende...","RSIA Resti Mulya, RS Harapan Jayakarta",1.0,1.0,NaN,NaN,Walikota Jakarta Timur,NaN,NaN
5451,https://www.rumah123.com/properti/jakarta-timu...,Dijual Rumah di Pulomas 2 Lantai SHMJalan depa...,2/18/2024 20:37,"Pulomas, Jakarta Timur",Lily tjahjadi,18 Februari 2024,"Rp 2,8 Miliar",5.0,5.0,136 m²,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5452,https://www.rumah123.com/properti/jakarta-timu...,Dijual Murah Rumah Siap Huni Laseine Lebar 6 H...,2/18/2024 20:37,"Cakung, Jakarta Timur",selvy,18 Februari 2024,"Rp 1,7 Miliar",3.0,2.0,102 m²,...,NaN,"Stasiun Cakung, Stasiun Buaran, Stasiun Klende...","RSIA Resti Mulya, RS Harapan Jayakarta",1.0,1.0,NaN,NaN,Walikota Jakarta Timur,NaN,NaN


# Data Preprocessing

### Rows and Columns

In [4]:
data_jaktim["wilayah"] = "Jakarta Timur"
data_jakpus["wilayah"] = "Jakarta Pusat"
data_jaksel["wilayah"] = "Jakarta Selatan"
data_jakbar["wilayah"] = "Jakarta Barat"
data_jakut["wilayah"] = "Jakarta Utara"

In [5]:
def get_kecamatan(region, kecamatan_dict):
    for kecamatan, daftar in kecamatan_dict.items():
        if any(r in region for r in daftar):
            return kecamatan
    return None

**Jakarta Timur**

Cakung

    - Cakung
    - Pulogebang
    - Penggilingan
Cipayung

    - Cipayung
    - Cilangkap
    - Bambu Apus
    - Lubang Buaya
    - Pondok Ranggon
    - Taman Mini
    - Setu
Ciracas

    - Cibubur
    - Kota Wisata
    - Citra Grand
    - Ciracas
    - Legenda Wisata
    - Raffles Hills
    - Kampung Rambutan
Duren Sawit

    - Duren Sawit
    - Pondok Kelapa
    - Pondok Bambu
    - Klender
    - Buaran
    - Pondok Kopi
    - Kalimalang
Jatinegara

    - Jatinegara
    - Otista
Kramat Jati

    - Condet
    - Kramat Jati
    - Cawang
    - Cililitan
    - Dewi Sartika
Makasar

    - Jatiwaringin
    - Pondok Gede
    - Cipinang Melayu
    - Halim Perdana Kusuma
    - Pinang Ranti
    - Makasar
    - Jati Cempaka
Matraman

    - Matraman
    - Utan Kayu
Pasar Rebo

    - Pasar Rebo
    - Kalisari
    - Cijantung
Pulo Gadung	

    - Rawamangun
    - Cipinang
    - Kayu Putih
    - Pulomas
    - Pulo Gadung
    - Pulo Asem
    - Pisangan Lama
    - Kampung Ambon
    - Kayu Jati
    - Ruko Rawa Lumbu

In [6]:
kecamatan_jaktim_dict = {
    'Cakung': ['Cakung', 'Pulogebang', 'Penggilingan'],
    'Cipayung': ['Cipayung', 'Cilangkap', 'Bambu Apus', 'Lubang Buaya', 'Pondok Ranggon', 'Taman Mini', 'Setu'],
    'Ciracas': ['Cibubur', 'Kota Wisata', 'Citra Grand', 'Ciracas', 'Legenda Wisata', 'Raffles Hills', 'Kampung Rambutan'],
    'Duren Sawit': ['Duren Sawit', 'Pondok Kelapa', 'Pondok Bambu', 'Klender', 'Buaran', 'Pondok Kopi', 'Kalimalang'],
    'Jatinegara': ['Jatinegara', 'Otista'],
    'Kramat Jati': ['Condet', 'Kramat Jati', 'Cawang', 'Cililitan', 'Dewi Sartika'],
    'Makasar': ['Jatiwaringin', 'Pondok Gede', 'Cipinang Melayu', 'Halim Perdana Kusuma', 'Pinang Ranti', 'Makasar', 'Jati Cempaka'],
    'Matraman': ['Matraman', 'Utan Kayu'],
    'Pasar Rebo': ['Pasar Rebo', 'Kalisari', 'Cijantung'],
    'Pulo Gadung': ['Rawamangun', 'Cipinang', 'Kayu Putih', 'Pulomas', 'Pulo Gadung', 'Pulo Asem', 'Pisangan Lama', 'Kampung Ambon', 'Kayu Jati', 'Ruko Rawa Lumbu']
}

In [7]:
data_jaktim['kecamatan'] = data_jaktim['region'].apply(lambda x: get_kecamatan(x, kecamatan_jaktim_dict))

In [8]:
data_jaktim["kecamatan"].value_counts()

Ciracas        1255
Duren Sawit    1167
Cakung         1114
Pulo Gadung     915
Cipayung        359
Kramat Jati     220
Makasar         120
Matraman        110
Pasar Rebo      104
Jatinegara       90
Name: kecamatan, dtype: int64

**Jakarta Pusat**

Kecamatan Cempaka Putih

    Cempaka Putih
    Cempaka Mas
    Percetakan Negara
    
Kecamatan Gambir

    Gambir
    Harmoni
    Gajah Mada
    Hayam Wuruk
    Petojo
    Batu Ceper
    Cideng
    Hasyim Ashari
    Batutulis
    
Kecamatan Johar Baru

    Johar Baru
    
Kecamatan Kemayoran

    Kemayoran
    Sumur Batu

Kecamatan Menteng

    Menteng
    Cikini
    Salemba
    Pegangsaan
    Gondangdia
    Wahid Hasyim
    Tambak Pegangsaan
    Menteng Atas
    Thamrin
    Kebon Sirih
    
Kecamatan Sawah Besar

    Sawah Besar
    Kartini
    Karang Anyar
    Mangga Dua
    Glodok
    Pasar Baru
    Samanhudi
    Pangeran Jayakarta
    Gunung Sahari
    Pinangsia

Kecamatan Senen

    Senen
    Kramat
    Bungur
    Salemba

Kecamatan Tanah Abang

    Tanah Abang
    Bendungan Hilir
    Karet Tengsin
    Pejompongan
    Kebon Kacang
    Kebon Melati
    KH Mas Mansyur
    Roxy
    CBD Area
    Senayan
    Bedungan Jatiluhur

In [9]:
kecamatan_dict_jakpus = {
    'Cempaka Putih': ['Cempaka Putih', 'Cempaka Mas', 'Percetakan Negara'],
    'Gambir': ['Gambir', 'Harmoni', 'Gajah Mada', 'Hayam Wuruk', 'Petojo', 'Batu Ceper', 'Cideng', 'Hasyim Ashari', 'Batutulis'],
    'Johar Baru': ['Johar Baru'],
    'Kemayoran': ['Kemayoran', 'Sumur Batu'],
    'Menteng': ['Menteng', 'Cikini', 'Salemba', 'Pegangsaan', 'Gondangdia', 'Wahid Hasyim', 'Tambak Pegangsaan', 'Menteng Atas', 'Thamrin', 'Kebon Sirih'],
    'Sawah Besar': ['Sawah Besar', 'Kartini', 'Karang Anyar', 'Mangga Dua', 'Glodok', 'Pasar Baru', 'Samanhudi', 'Pangeran Jayakarta', 'Gunung Sahari', 'Pinangsia'],
    'Senen': ['Senen', 'Kramat', 'Bungur', 'Salemba'],
    'Tanah Abang': ['Tanah Abang', 'Bendungan Hilir', 'Karet Tengsin', 'Pejompongan', 'Kebon Kacang', 'Kebon Melati', 'KH Mas Mansyur', 'Roxy', 'CBD Area', 'Senayan','Bedungan Jatiluhur'],
}

In [10]:
data_jakpus['kecamatan'] = data_jakpus['region'].apply(lambda x: get_kecamatan(x, kecamatan_dict_jakpus))

In [11]:
data_jakpus['kecamatan'].value_counts()

Menteng          1565
Cempaka Putih    1293
Gambir            636
Tanah Abang       471
Kemayoran         426
Sawah Besar       372
Senen             353
Johar Baru        136
Name: kecamatan, dtype: int64

**Jakarta Utara**

Kecamatan Cilincing

    Cilincing
    Rorotan
    Semper
    Marunda

Kecamatan Kelapa Gading

    Kelapa Gading
    Pegangsaan

Kecamatan Koja

    Koja
    Rawa Badak

Kecamatan Pademangan

    Pademangan
    Ancol

Kecamatan Penjaringan

    Pantai Indah Kapuk
    Pantai Indah Kapuk 2
    Muara Karang
    Pluit
    Penjaringan
    Pantai Mutiara
    Teluk Gong
    Kapuk Muara
    Kapuk
    Golf Island
    Taman Grisenda
    Bandengan
    Jembatan Tiga

Kecamatan Tanjung Priok

    Sunter
    Tanjung Priok
    Plumpang
    Kamal

In [12]:
kecamatan_dict_jakut = {
    'Cilincing': ['Cilincing', 'Rorotan', 'Semper', 'Marunda'],
    'Kelapa Gading': ['Kelapa Gading', 'Pegangsaan'],
    'Koja': ['Koja', 'Rawa Badak'],
    'Pademangan': ['Pademangan', 'Ancol'],
    'Penjaringan': ['Penjaringan','Pantai Indah Kapuk', 'Pantai Indah Kapuk 2', 'Muara Karang', 'Pluit', 'Pantai Mutiara', 'Teluk Gong', 'Kapuk Muara', 'Kapuk', 'Golf Island', 'Taman Grisenda', 'Bandengan', 'Jembatan Tiga'],
    'Tanjung Priok': ['Sunter', 'Tanjung Priok', 'Plumpang', 'Kamal']
}

In [13]:
data_jakut['kecamatan'] = data_jakut['region'].apply(lambda x: get_kecamatan(x, kecamatan_dict_jakut))

In [14]:
data_jakut['kecamatan'].value_counts()

Penjaringan      2806
Kelapa Gading    1525
Tanjung Priok     884
Pademangan        131
Cilincing          86
Koja               45
Name: kecamatan, dtype: int64

**Jakarta Barat**

Kecamatan Cengkareng

    Cengkareng
    Cengkareng Barat
    Duri Kosambi
    Kapuk Kamal
    Bojong Indah
    Taman Surya
    Taman Palem
    Green Lake City
    Taman Kencana
    Green Mansion
    Rawa Buaya
    Bandara

Kecamatan Grogol Petamburan

    Jelambar
    Tomang
    Tanjung Duren
    Tanjung Duren Selatan
    Tanjung Duren Utara
    Taman Anggrek
    Grogol
    Tawakal
    Jelambar Baru
    Jembatan Besi
    S Parman
    Tubagus Angke
    Gelong Baru
    Gelong

Kecamatan Taman Sari

    Tamansari
    Mangga Besar
    Kota
    Asemka
    Mangga Dua

Kecamatan Tambora

    Tambora
    Angke
    Jembatan Besi
    Jembatan Lima
    Jembatan Dua

Kecamatan Kebon Jeruk

    Kebon Jeruk
    Duri Kepa
    Green Ville
    Sunrise Garden
    Kedoya
    Kedoya Baru
    Kedoya Selatan
    Kedoya Utara
    Intercon
    Jalan Panjang
    Taman Ratu
    Green garden
    Pos Pengumben
    Alfa Indah
    Kepa Duri
    Kelapa Dua
    Taman Cosmos
    Rawa Belong

Kecamatan Kalideres

    Kalideres
    Citra Garden
    Semanan
    Daan Mogot
    Duta Garden
    Pegadungan

Kecamatan Palmerah

    Palmerah
    Kemanggisan
    Slipi
    Kota Bambu Utara

Kecamatan Kembangan

    Kembangan
    Kembangan Selatan
    Kembangan Baru
    Meruya
    Taman Meruya
    Taman Kota
    Taman Grisenda
    Pesanggrahan
    Puri Indah
    Permata Buana
    Puri Mansion
    Puri Media
    Metland Puri
    Kav DKI
    Joglo
    Srengseng
    Villa Meruya
    Kopilas
    Metro permata
    Karang Mulia
    Bisnis Park

In [15]:
kecamatan_dict_jakbar = {
    'Cengkareng': ['Cengkareng', 'Cengkareng Barat', 'Duri Kosambi', 'Kapuk Kamal', 'Bojong Indah', 'Taman Surya', 'Taman Palem', 'Green Lake City', 'Taman Kencana', 'Green Mansion', 'Rawa Buaya', 'Bandara'],
    'Grogol Petamburan': ['Jelambar', 'Tomang', 'Tanjung Duren', 'Tanjung Duren Selatan', 'Tanjung Duren Utara', 'Taman Anggrek', 'Grogol', 'Tawakal', 'Jelambar Baru', 'Jembatan Besi', 'S Parman', 'Tubagus Angke', 'Gelong Baru', 'Gelong'],
    'Taman Sari': ['Tamansari', 'Mangga Besar', 'Kota', 'Asemka', 'Mangga Dua'],
    'Tambora': ['Tambora', 'Angke', 'Jembatan Besi', 'Jembatan Lima', 'Jembatan Dua'],
    'Kebon Jeruk': ['Kebon Jeruk', 'Duri Kepa', 'Green Ville', 'Sunrise Garden', 'Kedoya', 'Kedoya Baru', 'Kedoya Selatan', 'Kedoya Utara', 'Intercon', 'Jalan Panjang', 'Taman Ratu', 'Green garden', 'Pos Pengumben', 'Alfa Indah', 'Kepa Duri', 'Kelapa Dua', 'Taman Cosmos', 'Rawa Belong'],
    'Kalideres': ['Kalideres', 'Citra Garden', 'Semanan', 'Daan Mogot', 'Duta Garden', 'Pegadungan'],
    'Palmerah': ['Palmerah', 'Kemanggisan', 'Slipi', 'Kota Bambu Utara'],
    'Kembangan': ['Kembangan', 'Kembangan Selatan', 'Kembangan Baru', 'Meruya', 'Taman Meruya', 'Taman Kota', 'Taman Grisenda', 'Pesanggrahan', 'Puri Indah', 'Permata Buana', 'Puri Mansion', 'Puri Media', 'Metland Puri', 'Kav DKI', 'Joglo', 'Srengseng', 'Villa Meruya', 'Kopilas', 'Metro permata', 'Karang Mulia', 'Bisnis Park']
}

In [16]:
data_jakbar['kecamatan'] = data_jakbar['region'].apply(lambda x: get_kecamatan(x, kecamatan_dict_jakbar))

In [17]:
data_jakbar['kecamatan'].value_counts()

Kembangan            1327
Grogol Petamburan    1292
Kebon Jeruk          1235
Cengkareng            807
Kalideres             576
Taman Sari             93
Palmerah               91
Tambora                74
Name: kecamatan, dtype: int64

**Jakarta Selatan**

Kecamatan Cilandak

    Cilandak
    Cipete
    Lebak Bulus
    Fatmawati
    TB Simatupang
    Cinere
    Cirendeu
    Pondok Labu
    Gandaria
    Terogong

Kecamatan Jagakarsa

    Jagakarsa
    Ciganjur
    Tanjung Barat
    Lenteng Agung
    Cipedak


Kecamatan Kebayoran Baru

    Kebayoran Baru
    Senopati
    Radio Dalam
    Panglima Polim
    Melawai
    Pakubuwono
    Blok M
    Senayan
    Rawa Belong
    Blok S
    Prapanca
    SCBD
    Antasari
    Wijaya

Kecamatan Kebayoran Lama

    Kebayoran Lama
    Kebayoran Lama Utara
    Kebayoran Lama Selatan
    Grogol Selatan
    Grogol Utara
    Cipulir
    Permata Hijau
    Pondok Pinang
    Simprug
    Tanah Kusir
    Patal Senayan
    Simprug Garden
    Praja Dalam
    Kemandoran

Kecamatan Mampang Prapatan

    Mampang Prapatan
    Mampang
    Tendean
    Kemang
    Bangka
    Kuningan
    Kapten Tendean

Kecamatan Pancoran

    Pancoran
    Pengadegan
    Cikoko
    Kebon Baru
    Kalibata
    Rawajati
    Duren Tiga
    Warung Buncit
    Gatot Subroto

Kecamatan Pasar Minggu

    Pasar Minggu
    Pejaten
    Cilandak Timur
    Cilandak Barat
    Pondok Indah
    Pondok Cabe
    Kebagusan
    Ampera
    Jati Padang
    Ragunan

Kecamatan Pesanggrahan

    Pesanggrahan
    Ulujami
    Petukangan
    Bintaro
    Veteran

Kecamatan Setiabudi

    Setiabudi
    Kuningan
    Mega Kuningan
    patra kuningan
    Karet
    Karet Kuningan
    Kampung Bali
    CBD
    Guntur
    Sudirman

Kecamatan Tebet

    Tebet
    Manggarai
    Menteng Dalam
    Tebet Timur
    Tebet Barat
    Karet Tengsin
    Karet Semanggi
    Bukit Duri
    Saharjo
    Gudang Peluru
    Cassablanca

In [18]:
kecamatan_dict_jaksel = {
    'Cilandak': ['Cilandak', 'Cipete', 'Lebak Bulus', 'Fatmawati', 'TB Simatupang', 'Cinere', 'Cirendeu', 'Pondok Labu', 'Gandaria', 'Terogong'],
    'Jagakarsa': ['Jagakarsa', 'Ciganjur', 'Tanjung Barat', 'Lenteng Agung', 'Cipedak'],
    'Kebayoran Baru': ['Kebayoran Baru', 'Senopati', 'Radio Dalam', 'Panglima Polim', 'Melawai', 'Pakubuwono', 'Blok M', 'Senayan', 'Rawa Belong', 'Blok S', 'Prapanca', 'SCBD', 'Antasari', 'Wijaya'],
    'Kebayoran Lama': ['Kebayoran Lama', 'Kebayoran Lama Utara', 'Kebayoran Lama Selatan', 'Grogol Selatan', 'Grogol Utara', 'Cipulir', 'Permata Hijau', 'Pondok Pinang', 'Simprug', 'Tanah Kusir', 'Patal Senayan', 'Simprug Garden', 'Praja Dalam', 'Kemandoran'],
    'Mampang Prapatan': ['Mampang Prapatan', 'Mampang', 'Tendean', 'Kemang', 'Bangka', 'Kuningan', 'Kapten Tendean'],
    'Pancoran': ['Pancoran', 'Pengadegan', 'Cikoko', 'Kebon Baru', 'Kalibata', 'Rawajati', 'Duren Tiga', 'Warung Buncit', 'Gatot Subroto'],
    'Pasar Minggu': ['Pasar Minggu', 'Pejaten', 'Cilandak Timur', 'Cilandak Barat', 'Pondok Indah', 'Pondok Cabe', 'Kebagusan', 'Ampera', 'Jati Padang', 'Ragunan'],
    'Pesanggrahan': ['Pesanggrahan', 'Ulujami', 'Petukangan', 'Bintaro', 'Veteran'],
    'Setiabudi': ['Setiabudi', 'Kuningan', 'Mega Kuningan', 'patra kuningan', 'Karet', 'Karet Kuningan', 'Kampung Bali', 'CBD', 'Guntur', 'Sudirman'],
    'Tebet': ['Tebet', 'Manggarai', 'Menteng Dalam', 'Tebet Timur', 'Tebet Barat', 'Karet Tengsin', 'Karet Semanggi', 'Bukit Duri', 'Saharjo', 'Gudang Peluru', 'Cassablanca']
}

In [19]:
data_jaksel['kecamatan'] = data_jaksel['region'].apply(lambda x: get_kecamatan(x, kecamatan_dict_jaksel))

In [20]:
data_jaksel['kecamatan'].value_counts()

Cilandak            1203
Pasar Minggu        1146
Kebayoran Baru       851
Mampang Prapatan     589
Jagakarsa            566
Pesanggrahan         399
Kebayoran Lama       280
Tebet                274
Pancoran             122
Setiabudi             62
Name: kecamatan, dtype: int64

In [21]:
merged_data = pd.concat([data_jaktim, data_jakpus, data_jaksel, data_jakbar, data_jakut], ignore_index=True)

In [22]:
merged_data

,source,title,date of collecting,region,seller's name,latest modified date,price,Kamar Tidur,Kamar Mandi,Luas Tanah,...,Kesehatan,Kamar Pembantu,Kamar Mandi Pembantu,Tahun Di Renovasi,Wisata,Others,Sekolah,Ibadah,wilayah,kecamatan
0,https://www.rumah123.com/properti/jakarta-timu...,Rumah Dijual msh Gress Dekat Aeon Mall Dan...,1/25/2024 14:23,"Cakung, Jakarta Timur",Lastry,04 Januari 2024,"Rp 2,2 Miliar",3.0,2.0,84 m²,...,NaN,NaN,NaN,NaN,NaN,Walikota Jakarta Timur,NaN,NaN,Jakarta Timur,Cakung
1,https://www.rumah123.com/properti/jakarta-timu...,RUMAH EXCLUSIVE 2 LANTAI LOKASI STRATEGIS DEKA...,1/25/2024 14:24,"Duren Sawit, Jakarta Timur",Erni Rahmawati,30 Desember 2023,"Rp 1,7 Miliar",3.0,2.0,118 m²,...,NaN,NaN,NaN,NaN,NaN,"Duren Sawit, Pondok Kopi Bekasi",NaN,NaN,Jakarta Timur,Duren Sawit
2,https://www.rumah123.com/properti/jakarta-timu...,Rumah semi kantor Asri di Jl. Cipinang Baru Ra...,1/25/2024 14:24,"Cipinang, Jakarta Timur",Wawan,25 Januari 2024,"Rp 2,47 Miliar",2.0,2.0,76 m²,...,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,Jakarta Timur,Pulo Gadung
3,https://www.rumah123.com/properti/jakarta-timu...,"Mau beli rumah? Ada loh dipisangan Jaktim,rmh ...",1/25/2024 14:24,"Matraman, Jakarta Timur",Debora,08 Januari 2024,Rp 828 Juta,2.0,2.0,31 m²,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jakarta Timur,Matraman
4,https://www.rumah123.com/properti/jakarta-timu...,Mau punya rumah di lok.strategis? Ada loh di j...,1/25/2024 14:24,"Pulo Gadung, Jakarta Timur",Debora,12 Januari 2024,"Rp 1,32 Miliar",3.0,2.0,48 m²,...,RS Persahabatan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jakarta Timur,Pulo Gadung
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27165,https://www.rumah123.com/properti/jakarta-utar...,"Danau sunter selatan 1000m, hadap jalan raya, ...",2/18/2024 17:43,"Sunter, Jakarta Utara",Tina Felani,11 Februari 2024,Rp 23 Miliar,4.0,4.0,1000 m²,...,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,Jakarta Utara,Tanjung Priok
27166,https://www.rumah123.com/properti/jakarta-utar...,"Gading kirana 9x17, 3lantai, tinggi dari jalan...",2/18/2024 17:43,"Kelapa Gading, Jakarta Utara",Tina Felani,11 Februari 2024,"Rp 5,7 Miliar",4.0,3.0,153 m²,...,"RS Mitra Keluarga Kelapa Gading, RS Gading Pluit",1.0,1.0,NaN,NaN,"Balai Samudera, Taman Jogging Kelapa Gading",SMAS Don Bosco 1,NaN,Jakarta Utara,Kelapa Gading
27167,https://www.rumah123.com/properti/jakarta-utar...,Dijual Rumah Cantik Hoek Dalam Komplek Di Pluit,2/18/2024 17:43,"Pluit, Jakarta Utara",edith,11 Februari 2024,"Rp 13,5 Miliar",5.0,4.0,512 m²,...,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,Jakarta Utara,Penjaringan
27168,https://www.rumah123.com/properti/jakarta-utar...,Rumah Cantik 2 Lantai Dalam Komplek Dekat Tol ...,2/18/2024 17:43,"Pluit, Jakarta Utara",edith,11 Februari 2024,Rp 6 Miliar,3.0,3.0,200 m²,...,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,Jakarta Utara,Penjaringan


In [23]:
merged_data["kecamatan"].value_counts()

Penjaringan          2806
Menteng              1565
Kelapa Gading        1525
Kembangan            1327
Cempaka Putih        1293
Grogol Petamburan    1292
Ciracas              1255
Kebon Jeruk          1235
Cilandak             1203
Duren Sawit          1167
Pasar Minggu         1146
Cakung               1114
Pulo Gadung           915
Tanjung Priok         884
Kebayoran Baru        851
Cengkareng            807
Gambir                636
Mampang Prapatan      589
Kalideres             576
Jagakarsa             566
Tanah Abang           471
Kemayoran             426
Pesanggrahan          399
Sawah Besar           372
Cipayung              359
Senen                 353
Kebayoran Lama        280
Tebet                 274
Kramat Jati           220
Johar Baru            136
Pademangan            131
Pancoran              122
Makasar               120
Matraman              110
Pasar Rebo            104
Taman Sari             93
Palmerah               91
Jatinegara             90
Cilincing   

### Check Duplicate Row

In [24]:
data_duplicate = merged_data.duplicated(subset=['ID Iklan']).sum()

print(f"Jumlah Baris yang duplikat {data_duplicate}")

Jumlah Baris yang duplikat 422


In [25]:
merged_data_unique = merged_data.drop_duplicates(subset=['ID Iklan'])

print(f"Jumlah baris yang duplikat setelah dilakukan penghapusan adalah {merged_data_unique.duplicated(subset=['ID Iklan']).sum()} baris")
print(f"Jumlah baris data adalah {merged_data_unique.shape[0]} baris")

Jumlah baris yang duplikat setelah dilakukan penghapusan adalah 0 baris
Jumlah baris data adalah 26748 baris


In [26]:
merged_data_unique.duplicated(subset=['ID Iklan']).sum()

0

In [27]:
merged_data_unique.shape

(26748, 47)

###  Penghapusan Kolom yang Tidak Relevan 

In [28]:
columns_to_drop = ["source","title","date of collecting",
                   "region","seller's name", "latest modified date","Tipe Properti","Tipe Iklan", "ID Iklan", "facilities"]
merged_data_unique.drop(columns=columns_to_drop, inplace=True)

C:\Users\Asus\AppData\Local\Temp\ipykernel_12408\3835243071.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data_unique.drop(columns=columns_to_drop, inplace=True)


In [29]:
merged_data_unique

,price,Kamar Tidur,Kamar Mandi,Luas Tanah,Luas Bangunan,Sertifikat,Daya Listrik,Dapur,Ruang Makan,Ruang Tamu,...,Kesehatan,Kamar Pembantu,Kamar Mandi Pembantu,Tahun Di Renovasi,Wisata,Others,Sekolah,Ibadah,wilayah,kecamatan
0,"Rp 2,2 Miliar",3.0,2.0,84 m²,79 m²,SHM - Sertifikat Hak Milik,2200 Watt,1.0,Tidak,Tidak,...,NaN,NaN,NaN,NaN,NaN,Walikota Jakarta Timur,NaN,NaN,Jakarta Timur,Cakung
1,"Rp 1,7 Miliar",3.0,2.0,118 m²,200 m²,SHM - Sertifikat Hak Milik,2200 Watt,1.0,Ya,Ya,...,NaN,NaN,NaN,NaN,NaN,"Duren Sawit, Pondok Kopi Bekasi",NaN,NaN,Jakarta Timur,Duren Sawit
2,"Rp 2,47 Miliar",2.0,2.0,76 m²,90 m²,SHM - Sertifikat Hak Milik,2200 Watt,1.0,Ya,Ya,...,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,Jakarta Timur,Pulo Gadung
3,Rp 828 Juta,2.0,2.0,31 m²,56 m²,SHM - Sertifikat Hak Milik,2200 Watt,1.0,Ya,Ya,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jakarta Timur,Matraman
4,"Rp 1,32 Miliar",3.0,2.0,48 m²,67 m²,SHM - Sertifikat Hak Milik,2200 Watt,1.0,Tidak,Tidak,...,RS Persahabatan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jakarta Timur,Pulo Gadung
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27165,Rp 23 Miliar,4.0,4.0,1000 m²,750 m²,HGB - Hak Guna Bangunan,NaN,NaN,NaN,NaN,...,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,Jakarta Utara,Tanjung Priok
27166,"Rp 5,7 Miliar",4.0,3.0,153 m²,225 m²,SHM - Sertifikat Hak Milik,4400 Watt,NaN,Tidak,Tidak,...,"RS Mitra Keluarga Kelapa Gading, RS Gading Pluit",1.0,1.0,NaN,NaN,"Balai Samudera, Taman Jogging Kelapa Gading",SMAS Don Bosco 1,NaN,Jakarta Utara,Kelapa Gading
27167,"Rp 13,5 Miliar",5.0,4.0,512 m²,300 m²,SHM - Sertifikat Hak Milik,7700 Watt,NaN,Tidak,Tidak,...,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,Jakarta Utara,Penjaringan
27168,Rp 6 Miliar,3.0,3.0,200 m²,275 m²,SHM - Sertifikat Hak Milik,NaN,NaN,Tidak,Tidak,...,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,Jakarta Utara,Penjaringan


### Data Cleaning 

Membersihkan nilai harga dengan menghilangkan 'Rp', spasi, dan melakukan konversi ke numeric

In [30]:
def convert_price_to_numeric(currency_str):
    # Menghilangkan karakter non-digit dari string
    numeric_str = currency_str.replace('Rp', '').replace(' ', '').replace(',','.')
    
    if 'Juta' in numeric_str:
        numeric_value = float(numeric_str.replace('Juta','')) * 10**6
    elif 'Miliar' in numeric_str:
        numeric_value = float(numeric_str.replace('Miliar','')) * 10**9
    elif 'Triliun' in numeric_str:
        numeric_value = float(numeric_str.replace('Triliun','')) * 10**12
    else:
        print("invalid")
        
    return numeric_value

In [31]:
merged_data_unique["price_before"]= merged_data_unique["price"]
merged_data_unique["price"] = merged_data_unique['price'].apply(convert_price_to_numeric)

C:\Users\Asus\AppData\Local\Temp\ipykernel_12408\1744785350.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data_unique["price_before"]= merged_data_unique["price"]
C:\Users\Asus\AppData\Local\Temp\ipykernel_12408\1744785350.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data_unique["price"] = merged_data_unique['price'].apply(convert_price_to_numeric)


In [32]:
merged_data_unique[["price_before","price"]]

,price_before,price
0,"Rp 2,2 Miliar",2.200000e+09
1,"Rp 1,7 Miliar",1.700000e+09
2,"Rp 2,47 Miliar",2.470000e+09
3,Rp 828 Juta,8.280000e+08
4,"Rp 1,32 Miliar",1.320000e+09
...,...,...
27165,Rp 23 Miliar,2.300000e+10
27166,"Rp 5,7 Miliar",5.700000e+09
27167,"Rp 13,5 Miliar",1.350000e+10
27168,Rp 6 Miliar,6.000000e+09


In [33]:
merged_data_unique.drop(columns=["price_before"], inplace=True)

C:\Users\Asus\AppData\Local\Temp\ipykernel_12408\984977658.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data_unique.drop(columns=["price_before"], inplace=True)


Menghilangkan satuan dari nilai variabel luas bangunan dan tanah

In [34]:
merged_data_unique["Luas Tanah_before"] = merged_data_unique["Luas Tanah"]
merged_data_unique["Luas Bangunan_before"] = merged_data_unique["Luas Bangunan"]
merged_data_unique["Luas Tanah"] = merged_data_unique["Luas Tanah"].str.replace(' m²', '').astype(float)
merged_data_unique["Luas Bangunan"] = merged_data_unique["Luas Bangunan"].str.replace(' m²', '').astype(float)

C:\Users\Asus\AppData\Local\Temp\ipykernel_12408\3522064284.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data_unique["Luas Tanah_before"] = merged_data_unique["Luas Tanah"]
C:\Users\Asus\AppData\Local\Temp\ipykernel_12408\3522064284.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data_unique["Luas Bangunan_before"] = merged_data_unique["Luas Bangunan"]
C:\Users\Asus\AppData\Local\Temp\ipykernel_12408\3522064284.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of 

In [35]:
merged_data_unique[["Luas Tanah_before","Luas Bangunan_before", "Luas Tanah", "Luas Bangunan"]]

,Luas Tanah_before,Luas Bangunan_before,Luas Tanah,Luas Bangunan
0,84 m²,79 m²,84.0,79.0
1,118 m²,200 m²,118.0,200.0
2,76 m²,90 m²,76.0,90.0
3,31 m²,56 m²,31.0,56.0
4,48 m²,67 m²,48.0,67.0
...,...,...,...,...
27165,1000 m²,750 m²,1000.0,750.0
27166,153 m²,225 m²,153.0,225.0
27167,512 m²,300 m²,512.0,300.0
27168,200 m²,275 m²,200.0,275.0


In [36]:
merged_data_unique.drop(columns=["Luas Tanah_before","Luas Bangunan_before"], inplace=True)

C:\Users\Asus\AppData\Local\Temp\ipykernel_12408\2863301392.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data_unique.drop(columns=["Luas Tanah_before","Luas Bangunan_before"], inplace=True)


Menghilangkan satuan dari nilai variabel Daya Listrik

In [37]:
merged_data_unique["Daya Listrik_before"] = merged_data_unique["Daya Listrik"]
merged_data_unique["Daya Listrik"] = merged_data_unique["Daya Listrik"].str.lower()
merged_data_unique["Daya Listrik"].replace("lainnya watt", pd.NA, inplace=True)
merged_data_unique["Daya Listrik"] = merged_data_unique["Daya Listrik"].str.replace(".", "").astype(float, errors='ignore')
merged_data_unique["Daya Listrik"] = merged_data_unique["Daya Listrik"].str.replace('watt', '', regex=False).astype(float, errors='ignore')

C:\Users\Asus\AppData\Local\Temp\ipykernel_12408\3772798197.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data_unique["Daya Listrik_before"] = merged_data_unique["Daya Listrik"]
C:\Users\Asus\AppData\Local\Temp\ipykernel_12408\3772798197.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data_unique["Daya Listrik"] = merged_data_unique["Daya Listrik"].str.lower()
C:\Users\Asus\AppData\Local\Temp\ipykernel_12408\3772798197.py:3: SettingWithCopyWarning: 
A value is trying to be set on a c

In [38]:
merged_data_unique[["Daya Listrik_before","Daya Listrik"]]

,Daya Listrik_before,Daya Listrik
0,2200 Watt,2200
1,2200 Watt,2200
2,2200 Watt,2200
3,2200 Watt,2200
4,2200 Watt,2200
...,...,...
27165,NaN,NaN
27166,4400 Watt,4400
27167,7700 Watt,7700
27168,NaN,NaN


In [39]:
merged_data_unique.drop(columns=["Daya Listrik_before"], inplace=True)

C:\Users\Asus\AppData\Local\Temp\ipykernel_12408\2671808886.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data_unique.drop(columns=["Daya Listrik_before"], inplace=True)


Menghilangkan satuan dari nilai variabel "Lebar Jalan"

In [40]:
merged_data_unique["Lebar Jalan_before"] = merged_data_unique["Lebar Jalan"]
merged_data_unique["Lebar Jalan"] = merged_data_unique["Lebar Jalan"].str.replace('Mobil',"").replace(" ","").astype(float, errors='ignore')

C:\Users\Asus\AppData\Local\Temp\ipykernel_12408\3327144976.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data_unique["Lebar Jalan_before"] = merged_data_unique["Lebar Jalan"]
C:\Users\Asus\AppData\Local\Temp\ipykernel_12408\3327144976.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data_unique["Lebar Jalan"] = merged_data_unique["Lebar Jalan"].str.replace('Mobil',"").replace(" ","").astype(float, errors='ignore')


In [41]:
merged_data_unique[["Lebar Jalan_before", "Lebar Jalan"]]

,Lebar Jalan_before,Lebar Jalan
0,2 Mobil,2.0
1,2 Mobil,2.0
2,2 Mobil,2.0
3,2 Mobil,2.0
4,1 Mobil,1.0
...,...,...
27165,NaN,NaN
27166,NaN,NaN
27167,NaN,NaN
27168,NaN,NaN


In [42]:
merged_data_unique.drop(columns=["Lebar Jalan_before"], inplace=True)

C:\Users\Asus\AppData\Local\Temp\ipykernel_12408\3424482090.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data_unique.drop(columns=["Lebar Jalan_before"], inplace=True)


Membuat nilai menjadi lowercase pada data kategori

In [43]:
merged_data_unique["Kondisi Properti_before"] = merged_data_unique["Kondisi Properti"] 
merged_data_unique["Kondisi Perabotan_before"] = merged_data_unique["Kondisi Perabotan"]

merged_data_unique["Kondisi Properti"] = merged_data_unique["Kondisi Properti"].str.lower()
merged_data_unique["Kondisi Perabotan"] = merged_data_unique["Kondisi Perabotan"].str.lower()

C:\Users\Asus\AppData\Local\Temp\ipykernel_12408\3794752950.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data_unique["Kondisi Properti_before"] = merged_data_unique["Kondisi Properti"]
C:\Users\Asus\AppData\Local\Temp\ipykernel_12408\3794752950.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data_unique["Kondisi Perabotan_before"] = merged_data_unique["Kondisi Perabotan"]
C:\Users\Asus\AppData\Local\Temp\ipykernel_12408\3794752950.py:4: SettingWithCopyWarning: 
A value is trying to 

In [44]:
merged_data_unique[["Kondisi Properti_before", "Kondisi Perabotan_before", "Kondisi Properti","Kondisi Perabotan"]]

,Kondisi Properti_before,Kondisi Perabotan_before,Kondisi Properti,Kondisi Perabotan
0,Baru,Unfurnished,baru,unfurnished
1,Baru,Unfurnished,baru,unfurnished
2,Bagus,Unfurnished,bagus,unfurnished
3,Bagus,Unfurnished,bagus,unfurnished
4,Baru,Unfurnished,baru,unfurnished
...,...,...,...,...
27165,Butuh Renovasi,Unfurnished,butuh renovasi,unfurnished
27166,Bagus,Unfurnished,bagus,unfurnished
27167,Bagus,Semi Furnished,bagus,semi furnished
27168,Bagus,Unfurnished,bagus,unfurnished


In [45]:
merged_data_unique.drop(columns=["Kondisi Properti_before", "Kondisi Perabotan_before"], inplace=True)

C:\Users\Asus\AppData\Local\Temp\ipykernel_12408\1330393289.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data_unique.drop(columns=["Kondisi Properti_before", "Kondisi Perabotan_before"], inplace=True)


### Check Missing Values

In [46]:
missing_value = merged_data_unique.isnull().sum()

In [47]:
columns_required = missing_value[missing_value <= merged_data_unique.shape[0]*0.5]
columns_required

price                      0
Kamar Tidur             1518
Kamar Mandi             1636
Luas Tanah                20
Luas Bangunan            120
Sertifikat               596
Daya Listrik            7136
Dapur                  11874
Ruang Makan             3217
Ruang Tamu              3120
Kondisi Perabotan       8096
Jumlah Lantai           1951
Terjangkau Internet     3121
Lebar Jalan            11930
Hook                    3121
Kondisi Properti        3109
Carport                11862
Sumber Air             10839
Kamar Pembantu         12809
wilayah                    0
kecamatan                  0
dtype: int64

In [48]:
merge_data_unique = merged_data_unique[list(columns_required.index)]

In [49]:
merge_data_unique

,price,Kamar Tidur,Kamar Mandi,Luas Tanah,Luas Bangunan,Sertifikat,Daya Listrik,Dapur,Ruang Makan,Ruang Tamu,...,Jumlah Lantai,Terjangkau Internet,Lebar Jalan,Hook,Kondisi Properti,Carport,Sumber Air,Kamar Pembantu,wilayah,kecamatan
0,2.200000e+09,3.0,2.0,84.0,79.0,SHM - Sertifikat Hak Milik,2200,1.0,Tidak,Tidak,...,2.0,Ya,2.0,Tidak,baru,2.0,PAM atau PDAM,NaN,Jakarta Timur,Cakung
1,1.700000e+09,3.0,2.0,118.0,200.0,SHM - Sertifikat Hak Milik,2200,1.0,Ya,Ya,...,2.0,Ya,2.0,Tidak,baru,1.0,Sumur Pompa,NaN,Jakarta Timur,Duren Sawit
2,2.470000e+09,2.0,2.0,76.0,90.0,SHM - Sertifikat Hak Milik,2200,1.0,Ya,Ya,...,2.0,Ya,2.0,Tidak,bagus,1.0,PAM atau PDAM,1.0,Jakarta Timur,Pulo Gadung
3,8.280000e+08,2.0,2.0,31.0,56.0,SHM - Sertifikat Hak Milik,2200,1.0,Ya,Ya,...,2.0,Ya,2.0,Tidak,bagus,1.0,PAM atau PDAM,NaN,Jakarta Timur,Matraman
4,1.320000e+09,3.0,2.0,48.0,67.0,SHM - Sertifikat Hak Milik,2200,1.0,Tidak,Tidak,...,2.0,Tidak,1.0,Tidak,baru,1.0,PAM atau PDAM,NaN,Jakarta Timur,Pulo Gadung
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27165,2.300000e+10,4.0,4.0,1000.0,750.0,HGB - Hak Guna Bangunan,NaN,NaN,NaN,NaN,...,2.0,NaN,NaN,NaN,butuh renovasi,NaN,NaN,1.0,Jakarta Utara,Tanjung Priok
27166,5.700000e+09,4.0,3.0,153.0,225.0,SHM - Sertifikat Hak Milik,4400,NaN,Tidak,Tidak,...,3.0,Tidak,NaN,Tidak,bagus,NaN,NaN,1.0,Jakarta Utara,Kelapa Gading
27167,1.350000e+10,5.0,4.0,512.0,300.0,SHM - Sertifikat Hak Milik,7700,NaN,Tidak,Tidak,...,2.0,Tidak,NaN,Ya,bagus,2.0,PAM atau PDAM,1.0,Jakarta Utara,Penjaringan
27168,6.000000e+09,3.0,3.0,200.0,275.0,SHM - Sertifikat Hak Milik,NaN,NaN,Tidak,Tidak,...,2.0,Tidak,NaN,Tidak,bagus,2.0,NaN,1.0,Jakarta Utara,Penjaringan


In [50]:
merge_data_unique["Daya Listrik"] = merge_data_unique["Daya Listrik"].astype(float, errors='ignore')

C:\Users\Asus\AppData\Local\Temp\ipykernel_12408\1450367074.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merge_data_unique["Daya Listrik"] = merge_data_unique["Daya Listrik"].astype(float, errors='ignore')


In [51]:
merge_data_unique["Kamar Tidur"].fillna(round(merge_data_unique["Kamar Tidur"].mean()),inplace=True)
merge_data_unique["Kamar Mandi"].fillna(round(merge_data_unique["Kamar Mandi"].mean()),inplace=True)
merge_data_unique["Luas Tanah"].fillna(merge_data_unique["Luas Tanah"].mean(),inplace=True)
merge_data_unique["Luas Bangunan"].fillna(merge_data_unique["Luas Bangunan"].mean(),inplace=True)
merge_data_unique["Sertifikat"].fillna(merge_data_unique["Sertifikat"].mode()[0],inplace=True)
merge_data_unique["Daya Listrik"].fillna(merge_data_unique["Daya Listrik"].dropna().astype('float').mean(),inplace=True)
merge_data_unique["Dapur"].fillna(merge_data_unique["Dapur"].mode()[0],inplace=True)
merge_data_unique["Ruang Makan"].fillna(merge_data_unique["Ruang Makan"].mode()[0],inplace=True)
merge_data_unique["Ruang Tamu"].fillna(merge_data_unique["Ruang Tamu"].mode()[0],inplace=True)
merge_data_unique["Kondisi Perabotan"].fillna(merge_data_unique["Kondisi Perabotan"].mode()[0],inplace=True)
merge_data_unique["Jumlah Lantai"].fillna(round(merge_data_unique["Jumlah Lantai"].mean()),inplace=True)
merge_data_unique["Terjangkau Internet"].fillna(merge_data_unique["Terjangkau Internet"].mode()[0],inplace=True)
merge_data_unique["Lebar Jalan"].fillna(round(merge_data_unique["Lebar Jalan"].mean()),inplace=True)
merge_data_unique["Hook"].fillna(merge_data_unique["Hook"].mode()[0],inplace=True)
merge_data_unique["Kondisi Properti"].fillna(merge_data_unique["Kondisi Properti"].mode()[0],inplace=True)
merge_data_unique["Carport"].fillna(round(merge_data_unique["Carport"].mean()),inplace=True)
merge_data_unique["Sumber Air"].fillna(merge_data_unique["Sumber Air"].mode()[0],inplace=True)
merge_data_unique["Kamar Pembantu"].fillna(round(merge_data_unique["Kamar Pembantu"].mean()),inplace=True)

C:\Users\Asus\AppData\Local\Temp\ipykernel_12408\3001794162.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merge_data_unique["Kamar Tidur"].fillna(round(merge_data_unique["Kamar Tidur"].mean()),inplace=True)
C:\Users\Asus\AppData\Local\Temp\ipykernel_12408\3001794162.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merge_data_unique["Kamar Mandi"].fillna(round(merge_data_unique["Kamar Mandi"].mean()),inplace=True)
C:\Users\Asus\AppData\Local\Temp\ipykernel_12408\3001794162.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http

In [52]:
merge_data_unique.isna().sum()

price                  0
Kamar Tidur            0
Kamar Mandi            0
Luas Tanah             0
Luas Bangunan          0
Sertifikat             0
Daya Listrik           0
Dapur                  0
Ruang Makan            0
Ruang Tamu             0
Kondisi Perabotan      0
Jumlah Lantai          0
Terjangkau Internet    0
Lebar Jalan            0
Hook                   0
Kondisi Properti       0
Carport                0
Sumber Air             0
Kamar Pembantu         0
wilayah                0
kecamatan              0
dtype: int64

In [53]:
merge_data_unique

,price,Kamar Tidur,Kamar Mandi,Luas Tanah,Luas Bangunan,Sertifikat,Daya Listrik,Dapur,Ruang Makan,Ruang Tamu,...,Jumlah Lantai,Terjangkau Internet,Lebar Jalan,Hook,Kondisi Properti,Carport,Sumber Air,Kamar Pembantu,wilayah,kecamatan
0,2.200000e+09,3.0,2.0,84.0,79.0,SHM - Sertifikat Hak Milik,2200,1.0,Tidak,Tidak,...,2.0,Ya,2.0,Tidak,baru,2.0,PAM atau PDAM,1.0,Jakarta Timur,Cakung
1,1.700000e+09,3.0,2.0,118.0,200.0,SHM - Sertifikat Hak Milik,2200,1.0,Ya,Ya,...,2.0,Ya,2.0,Tidak,baru,1.0,Sumur Pompa,1.0,Jakarta Timur,Duren Sawit
2,2.470000e+09,2.0,2.0,76.0,90.0,SHM - Sertifikat Hak Milik,2200,1.0,Ya,Ya,...,2.0,Ya,2.0,Tidak,bagus,1.0,PAM atau PDAM,1.0,Jakarta Timur,Pulo Gadung
3,8.280000e+08,2.0,2.0,31.0,56.0,SHM - Sertifikat Hak Milik,2200,1.0,Ya,Ya,...,2.0,Ya,2.0,Tidak,bagus,1.0,PAM atau PDAM,1.0,Jakarta Timur,Matraman
4,1.320000e+09,3.0,2.0,48.0,67.0,SHM - Sertifikat Hak Milik,2200,1.0,Tidak,Tidak,...,2.0,Tidak,1.0,Tidak,baru,1.0,PAM atau PDAM,1.0,Jakarta Timur,Pulo Gadung
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27165,2.300000e+10,4.0,4.0,1000.0,750.0,HGB - Hak Guna Bangunan,5483.247502,1.0,Ya,Ya,...,2.0,Ya,2.0,Tidak,butuh renovasi,2.0,PAM atau PDAM,1.0,Jakarta Utara,Tanjung Priok
27166,5.700000e+09,4.0,3.0,153.0,225.0,SHM - Sertifikat Hak Milik,4400,1.0,Tidak,Tidak,...,3.0,Tidak,2.0,Tidak,bagus,2.0,PAM atau PDAM,1.0,Jakarta Utara,Kelapa Gading
27167,1.350000e+10,5.0,4.0,512.0,300.0,SHM - Sertifikat Hak Milik,7700,1.0,Tidak,Tidak,...,2.0,Tidak,2.0,Ya,bagus,2.0,PAM atau PDAM,1.0,Jakarta Utara,Penjaringan
27168,6.000000e+09,3.0,3.0,200.0,275.0,SHM - Sertifikat Hak Milik,5483.247502,1.0,Tidak,Tidak,...,2.0,Tidak,2.0,Tidak,bagus,2.0,PAM atau PDAM,1.0,Jakarta Utara,Penjaringan


### Data Transformation

In [54]:
merge_data_unique["Sertifikat"].value_counts()

SHM - Sertifikat Hak Milik       23940
Lainnya (PPJB,Girik,Adat,dll)     1455
HGB - Hak Guna Bangunan           1318
HP - Hak Pakai                      31
HS - Hak Sewa                        4
Name: Sertifikat, dtype: int64

In [55]:
one_hot_encoded_sertifikat = pd.get_dummies(merge_data_unique["Sertifikat"])

In [56]:
one_hot_encoded_sertifikat

,HGB - Hak Guna Bangunan,HP - Hak Pakai,HS - Hak Sewa,"Lainnya (PPJB,Girik,Adat,dll)",SHM - Sertifikat Hak Milik
0,0,0,0,0,1
1,0,0,0,0,1
2,0,0,0,0,1
3,0,0,0,0,1
4,0,0,0,0,1
...,...,...,...,...,...
27165,1,0,0,0,0
27166,0,0,0,0,1
27167,0,0,0,0,1
27168,0,0,0,0,1


In [57]:
merge_data_unique = pd.concat([merge_data_unique, one_hot_encoded_sertifikat], axis=1)

In [58]:
merge_data_unique.drop(columns=["Sertifikat"], inplace=True)

In [59]:
merge_data_unique

,price,Kamar Tidur,Kamar Mandi,Luas Tanah,Luas Bangunan,Daya Listrik,Dapur,Ruang Makan,Ruang Tamu,Kondisi Perabotan,...,Carport,Sumber Air,Kamar Pembantu,wilayah,kecamatan,HGB - Hak Guna Bangunan,HP - Hak Pakai,HS - Hak Sewa,"Lainnya (PPJB,Girik,Adat,dll)",SHM - Sertifikat Hak Milik
0,2.200000e+09,3.0,2.0,84.0,79.0,2200,1.0,Tidak,Tidak,unfurnished,...,2.0,PAM atau PDAM,1.0,Jakarta Timur,Cakung,0,0,0,0,1
1,1.700000e+09,3.0,2.0,118.0,200.0,2200,1.0,Ya,Ya,unfurnished,...,1.0,Sumur Pompa,1.0,Jakarta Timur,Duren Sawit,0,0,0,0,1
2,2.470000e+09,2.0,2.0,76.0,90.0,2200,1.0,Ya,Ya,unfurnished,...,1.0,PAM atau PDAM,1.0,Jakarta Timur,Pulo Gadung,0,0,0,0,1
3,8.280000e+08,2.0,2.0,31.0,56.0,2200,1.0,Ya,Ya,unfurnished,...,1.0,PAM atau PDAM,1.0,Jakarta Timur,Matraman,0,0,0,0,1
4,1.320000e+09,3.0,2.0,48.0,67.0,2200,1.0,Tidak,Tidak,unfurnished,...,1.0,PAM atau PDAM,1.0,Jakarta Timur,Pulo Gadung,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27165,2.300000e+10,4.0,4.0,1000.0,750.0,5483.247502,1.0,Ya,Ya,unfurnished,...,2.0,PAM atau PDAM,1.0,Jakarta Utara,Tanjung Priok,1,0,0,0,0
27166,5.700000e+09,4.0,3.0,153.0,225.0,4400,1.0,Tidak,Tidak,unfurnished,...,2.0,PAM atau PDAM,1.0,Jakarta Utara,Kelapa Gading,0,0,0,0,1
27167,1.350000e+10,5.0,4.0,512.0,300.0,7700,1.0,Tidak,Tidak,semi furnished,...,2.0,PAM atau PDAM,1.0,Jakarta Utara,Penjaringan,0,0,0,0,1
27168,6.000000e+09,3.0,3.0,200.0,275.0,5483.247502,1.0,Tidak,Tidak,unfurnished,...,2.0,PAM atau PDAM,1.0,Jakarta Utara,Penjaringan,0,0,0,0,1


In [60]:
merge_data_unique["Ruang Makan"].value_counts()

Ya       17363
Tidak     9385
Name: Ruang Makan, dtype: int64

In [61]:
# Define the mapping dictionary
mapping_ruang_makan = {"Ya": 1, "Tidak": 0}

# Map the values in the "Kondisi Perabotan" column using the mapping dictionary
merge_data_unique["Ruang Makan"] = merge_data_unique["Ruang Makan"].map(mapping_ruang_makan)

In [62]:
pd.DataFrame(merge_data_unique["Ruang Makan"])

,Ruang Makan
0,0
1,1
2,1
3,1
4,0
...,...
27165,1
27166,0
27167,0
27168,0


In [63]:
# Define the mapping dictionary
mapping_ruang_tamu = {"Ya": 1, "Tidak": 0}

# Map the values in the "Kondisi Perabotan" column using the mapping dictionary
merge_data_unique["Ruang Tamu"] = merge_data_unique["Ruang Tamu"].map(mapping_ruang_tamu)

In [64]:
pd.DataFrame(merge_data_unique["Ruang Tamu"])

,Ruang Tamu
0,0
1,1
2,1
3,1
4,0
...,...
27165,1
27166,0
27167,0
27168,0


In [65]:
merge_data_unique

,price,Kamar Tidur,Kamar Mandi,Luas Tanah,Luas Bangunan,Daya Listrik,Dapur,Ruang Makan,Ruang Tamu,Kondisi Perabotan,...,Carport,Sumber Air,Kamar Pembantu,wilayah,kecamatan,HGB - Hak Guna Bangunan,HP - Hak Pakai,HS - Hak Sewa,"Lainnya (PPJB,Girik,Adat,dll)",SHM - Sertifikat Hak Milik
0,2.200000e+09,3.0,2.0,84.0,79.0,2200,1.0,0,0,unfurnished,...,2.0,PAM atau PDAM,1.0,Jakarta Timur,Cakung,0,0,0,0,1
1,1.700000e+09,3.0,2.0,118.0,200.0,2200,1.0,1,1,unfurnished,...,1.0,Sumur Pompa,1.0,Jakarta Timur,Duren Sawit,0,0,0,0,1
2,2.470000e+09,2.0,2.0,76.0,90.0,2200,1.0,1,1,unfurnished,...,1.0,PAM atau PDAM,1.0,Jakarta Timur,Pulo Gadung,0,0,0,0,1
3,8.280000e+08,2.0,2.0,31.0,56.0,2200,1.0,1,1,unfurnished,...,1.0,PAM atau PDAM,1.0,Jakarta Timur,Matraman,0,0,0,0,1
4,1.320000e+09,3.0,2.0,48.0,67.0,2200,1.0,0,0,unfurnished,...,1.0,PAM atau PDAM,1.0,Jakarta Timur,Pulo Gadung,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27165,2.300000e+10,4.0,4.0,1000.0,750.0,5483.247502,1.0,1,1,unfurnished,...,2.0,PAM atau PDAM,1.0,Jakarta Utara,Tanjung Priok,1,0,0,0,0
27166,5.700000e+09,4.0,3.0,153.0,225.0,4400,1.0,0,0,unfurnished,...,2.0,PAM atau PDAM,1.0,Jakarta Utara,Kelapa Gading,0,0,0,0,1
27167,1.350000e+10,5.0,4.0,512.0,300.0,7700,1.0,0,0,semi furnished,...,2.0,PAM atau PDAM,1.0,Jakarta Utara,Penjaringan,0,0,0,0,1
27168,6.000000e+09,3.0,3.0,200.0,275.0,5483.247502,1.0,0,0,unfurnished,...,2.0,PAM atau PDAM,1.0,Jakarta Utara,Penjaringan,0,0,0,0,1


In [66]:
merge_data_unique["Kondisi Perabotan"].value_counts()

unfurnished       18862
semi furnished     5656
furnished          2230
Name: Kondisi Perabotan, dtype: int64

In [67]:
# Define the mapping dictionary
mapping_kondisi_perabotan = {"unfurnished": 0, "semi furnished": 1, "furnished": 2}

# Map the values in the "Kondisi Perabotan" column using the mapping dictionary
merge_data_unique["Kondisi Perabotan"] = merge_data_unique["Kondisi Perabotan"].map(mapping_kondisi_perabotan)

In [68]:
pd.DataFrame(merge_data_unique["Kondisi Perabotan"])

,Kondisi Perabotan
0,0
1,0
2,0
3,0
4,0
...,...
27165,0
27166,0
27167,1
27168,0


In [69]:
merge_data_unique["Terjangkau Internet"].value_counts()

Ya       18358
Tidak     8390
Name: Terjangkau Internet, dtype: int64

In [70]:
# Define the mapping dictionary
mapping_terjangkau_internet = {"Ya": 1, "Tidak": 0}

# Map the values in the "Kondisi Perabotan" column using the mapping dictionary
merge_data_unique["Terjangkau Internet"] = merge_data_unique["Terjangkau Internet"].map(mapping_terjangkau_internet)

In [71]:
pd.DataFrame(merge_data_unique["Terjangkau Internet"])

,Terjangkau Internet
0,1
1,1
2,1
3,1
4,0
...,...
27165,1
27166,0
27167,0
27168,0


In [72]:
merge_data_unique["Hook"].value_counts()

Tidak    24195
Ya        2553
Name: Hook, dtype: int64

In [73]:
# Define the mapping dictionary
mapping_hook = {"Ya": 1, "Tidak": 0}

# Map the values in the "Kondisi Perabotan" column using the mapping dictionary
merge_data_unique["Hook"] = merge_data_unique["Hook"].map(mapping_hook)

In [74]:
pd.DataFrame(merge_data_unique["Hook"])

,Hook
0,0
1,0
2,0
3,0
4,0
...,...
27165,0
27166,0
27167,1
27168,0


In [75]:
merge_data_unique["Kondisi Properti"].value_counts()

bagus             18650
baru               4345
butuh renovasi     2022
sudah renovasi     1192
bagus sekali        539
Name: Kondisi Properti, dtype: int64

In [76]:
# Define the mapping dictionary
mapping_kondisi_properti = {"butuh renovasi": 0, "sudah renovasi": 1, "baru": 2, "bagus": 3, "bagus sekali":4}

# Map the values in the "Kondisi Perabotan" column using the mapping dictionary
merge_data_unique["Kondisi Properti"] = merge_data_unique["Kondisi Properti"].map(mapping_kondisi_properti)

In [77]:
pd.DataFrame(merge_data_unique["Kondisi Properti"])

,Kondisi Properti
0,2
1,2
2,3
3,3
4,2
...,...
27165,0
27166,3
27167,3
27168,3


In [78]:
merge_data_unique["Sumber Air"].value_counts()

PAM atau PDAM    21517
Sumur Bor         2967
Sumur Pompa       2092
Sumur Resapan      129
Sumur Galian        43
Name: Sumber Air, dtype: int64

In [79]:
one_hot_encoded_sumber_air = pd.get_dummies(merge_data_unique["Sumber Air"])

In [80]:
one_hot_encoded_sumber_air

,PAM atau PDAM,Sumur Bor,Sumur Galian,Sumur Pompa,Sumur Resapan
0,1,0,0,0,0
1,0,0,0,1,0
2,1,0,0,0,0
3,1,0,0,0,0
4,1,0,0,0,0
...,...,...,...,...,...
27165,1,0,0,0,0
27166,1,0,0,0,0
27167,1,0,0,0,0
27168,1,0,0,0,0


In [81]:
merge_data_unique = pd.concat([merge_data_unique, one_hot_encoded_sumber_air], axis=1)

In [82]:
merge_data_unique

,price,Kamar Tidur,Kamar Mandi,Luas Tanah,Luas Bangunan,Daya Listrik,Dapur,Ruang Makan,Ruang Tamu,Kondisi Perabotan,...,HGB - Hak Guna Bangunan,HP - Hak Pakai,HS - Hak Sewa,"Lainnya (PPJB,Girik,Adat,dll)",SHM - Sertifikat Hak Milik,PAM atau PDAM,Sumur Bor,Sumur Galian,Sumur Pompa,Sumur Resapan
0,2.200000e+09,3.0,2.0,84.0,79.0,2200,1.0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
1,1.700000e+09,3.0,2.0,118.0,200.0,2200,1.0,1,1,0,...,0,0,0,0,1,0,0,0,1,0
2,2.470000e+09,2.0,2.0,76.0,90.0,2200,1.0,1,1,0,...,0,0,0,0,1,1,0,0,0,0
3,8.280000e+08,2.0,2.0,31.0,56.0,2200,1.0,1,1,0,...,0,0,0,0,1,1,0,0,0,0
4,1.320000e+09,3.0,2.0,48.0,67.0,2200,1.0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27165,2.300000e+10,4.0,4.0,1000.0,750.0,5483.247502,1.0,1,1,0,...,1,0,0,0,0,1,0,0,0,0
27166,5.700000e+09,4.0,3.0,153.0,225.0,4400,1.0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
27167,1.350000e+10,5.0,4.0,512.0,300.0,7700,1.0,0,0,1,...,0,0,0,0,1,1,0,0,0,0
27168,6.000000e+09,3.0,3.0,200.0,275.0,5483.247502,1.0,0,0,0,...,0,0,0,0,1,1,0,0,0,0


In [83]:
merge_data_unique.drop(columns=["Sumber Air"], inplace=True)

In [84]:
merge_data_unique["wilayah"].value_counts()

Jakarta Barat      5492
Jakarta Utara      5475
Jakarta Timur      5454
Jakarta Selatan    5344
Jakarta Pusat      4983
Name: wilayah, dtype: int64

In [85]:
one_hot_encoded_wilayah = pd.get_dummies(merge_data_unique["wilayah"])

In [86]:
one_hot_encoded_wilayah

,Jakarta Barat,Jakarta Pusat,Jakarta Selatan,Jakarta Timur,Jakarta Utara
0,0,0,0,1,0
1,0,0,0,1,0
2,0,0,0,1,0
3,0,0,0,1,0
4,0,0,0,1,0
...,...,...,...,...,...
27165,0,0,0,0,1
27166,0,0,0,0,1
27167,0,0,0,0,1
27168,0,0,0,0,1


In [87]:
merge_data_unique = pd.concat([merge_data_unique, one_hot_encoded_wilayah], axis=1)

In [88]:
merge_data_unique.drop(columns=["wilayah"], inplace=True)

In [89]:
one_hot_encoded_kecamatan = pd.get_dummies(merge_data_unique["kecamatan"])

In [90]:
one_hot_encoded_kecamatan

,Cakung,Cempaka Putih,Cengkareng,Cilandak,Cilincing,Cipayung,Ciracas,Duren Sawit,Gambir,Grogol Petamburan,...,Pesanggrahan,Pulo Gadung,Sawah Besar,Senen,Setiabudi,Taman Sari,Tambora,Tanah Abang,Tanjung Priok,Tebet
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27165,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
27166,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27167,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27168,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [91]:
merge_data_unique = pd.concat([merge_data_unique, one_hot_encoded_kecamatan], axis=1)

In [92]:
merge_data_unique.drop(columns=["kecamatan"], inplace=True)

In [93]:
merge_data_unique.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26748 entries, 0 to 27169
Data columns (total 74 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   price                          26748 non-null  float64
 1   Kamar Tidur                    26748 non-null  float64
 2   Kamar Mandi                    26748 non-null  float64
 3   Luas Tanah                     26748 non-null  float64
 4   Luas Bangunan                  26748 non-null  float64
 5   Daya Listrik                   26748 non-null  object 
 6   Dapur                          26748 non-null  float64
 7   Ruang Makan                    26748 non-null  int64  
 8   Ruang Tamu                     26748 non-null  int64  
 9   Kondisi Perabotan              26748 non-null  int64  
 10  Jumlah Lantai                  26748 non-null  float64
 11  Terjangkau Internet            26748 non-null  int64  
 12  Lebar Jalan                    26748 non-null 

In [94]:
merge_data_unique["Daya Listrik"] = merge_data_unique["Daya Listrik"].astype('float')

In [95]:
merge_data_unique.columns

Index(['price', 'Kamar Tidur', 'Kamar Mandi', 'Luas Tanah', 'Luas Bangunan',
       'Daya Listrik', 'Dapur', 'Ruang Makan', 'Ruang Tamu',
       'Kondisi Perabotan', 'Jumlah Lantai', 'Terjangkau Internet',
       'Lebar Jalan', 'Hook', 'Kondisi Properti', 'Carport', 'Kamar Pembantu',
       'HGB - Hak Guna Bangunan', 'HP - Hak Pakai', 'HS - Hak Sewa',
       'Lainnya (PPJB,Girik,Adat,dll)', 'SHM - Sertifikat Hak Milik',
       'PAM atau PDAM', 'Sumur Bor', 'Sumur Galian', 'Sumur Pompa',
       'Sumur Resapan', 'Jakarta Barat', 'Jakarta Pusat', 'Jakarta Selatan',
       'Jakarta Timur', 'Jakarta Utara', 'Cakung', 'Cempaka Putih',
       'Cengkareng', 'Cilandak', 'Cilincing', 'Cipayung', 'Ciracas',
       'Duren Sawit', 'Gambir', 'Grogol Petamburan', 'Jagakarsa', 'Jatinegara',
       'Johar Baru', 'Kalideres', 'Kebayoran Baru', 'Kebayoran Lama',
       'Kebon Jeruk', 'Kelapa Gading', 'Kemayoran', 'Kembangan', 'Koja',
       'Kramat Jati', 'Makasar', 'Mampang Prapatan', 'Matraman', 'Mente

In [96]:
merge_data_unique

,price,Kamar Tidur,Kamar Mandi,Luas Tanah,Luas Bangunan,Daya Listrik,Dapur,Ruang Makan,Ruang Tamu,Kondisi Perabotan,...,Pesanggrahan,Pulo Gadung,Sawah Besar,Senen,Setiabudi,Taman Sari,Tambora,Tanah Abang,Tanjung Priok,Tebet
0,2.200000e+09,3.0,2.0,84.0,79.0,2200.000000,1.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1.700000e+09,3.0,2.0,118.0,200.0,2200.000000,1.0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,2.470000e+09,2.0,2.0,76.0,90.0,2200.000000,1.0,1,1,0,...,0,1,0,0,0,0,0,0,0,0
3,8.280000e+08,2.0,2.0,31.0,56.0,2200.000000,1.0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4,1.320000e+09,3.0,2.0,48.0,67.0,2200.000000,1.0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27165,2.300000e+10,4.0,4.0,1000.0,750.0,5483.247502,1.0,1,1,0,...,0,0,0,0,0,0,0,0,1,0
27166,5.700000e+09,4.0,3.0,153.0,225.0,4400.000000,1.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27167,1.350000e+10,5.0,4.0,512.0,300.0,7700.000000,1.0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
27168,6.000000e+09,3.0,3.0,200.0,275.0,5483.247502,1.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [97]:
merge_data_unique.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26748 entries, 0 to 27169
Data columns (total 74 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   price                          26748 non-null  float64
 1   Kamar Tidur                    26748 non-null  float64
 2   Kamar Mandi                    26748 non-null  float64
 3   Luas Tanah                     26748 non-null  float64
 4   Luas Bangunan                  26748 non-null  float64
 5   Daya Listrik                   26748 non-null  float64
 6   Dapur                          26748 non-null  float64
 7   Ruang Makan                    26748 non-null  int64  
 8   Ruang Tamu                     26748 non-null  int64  
 9   Kondisi Perabotan              26748 non-null  int64  
 10  Jumlah Lantai                  26748 non-null  float64
 11  Terjangkau Internet            26748 non-null  int64  
 12  Lebar Jalan                    26748 non-null 

### Outlier

In [121]:
# Calculate Q1 (25th percentile) and Q3 (75th percentile)
Q1 = merge_data_unique["Kamar Tidur"].quantile(0.25)
Q3 = merge_data_unique["Kamar Tidur"].quantile(0.75)

# Calculate the IQR
IQR = Q3 - Q1

# Define the lower and upper bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

In [122]:
outliers = merge_data_unique[(merge_data_unique["Kamar Tidur"] < lower_bound) | (merge_data_unique["Kamar Tidur"] > upper_bound)]
outliers

,price,Kamar Tidur,Kamar Mandi,Luas Tanah,Luas Bangunan,Daya Listrik,Dapur,Ruang Makan,Ruang Tamu,Kondisi Perabotan,...,Pesanggrahan,Pulo Gadung,Sawah Besar,Senen,Setiabudi,Taman Sari,Tambora,Tanah Abang,Tanjung Priok,Tebet
100,1.650000e+10,54.0,54.0,570.0,1500.0,53000.000000,2.0,1,1,2,...,0,0,0,0,0,0,0,0,0,0
144,2.500000e+10,10.0,2.0,632.0,271.0,5483.247502,1.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
272,1.200000e+10,9.0,8.0,1276.0,863.0,13900.000000,2.0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
299,2.550000e+09,13.0,3.0,155.0,265.0,2200.000000,2.0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
408,4.600000e+09,11.0,11.0,290.0,525.0,13200.000000,2.0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26854,2.750000e+09,9.0,9.0,168.0,168.0,3300.000000,1.0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
26938,1.500000e+10,9.0,6.0,649.0,500.0,5483.247502,1.0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
26988,3.950000e+09,9.0,5.0,140.0,340.0,5483.247502,1.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27093,4.280000e+09,20.0,17.0,190.0,500.0,6600.000000,1.0,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [123]:
len(outliers)

1165

In [239]:
len(merge_data_unique)

26748

In [55]:
# Handle outliers by capping and flooring
merge_data_unique["Kamar Tidur"] = np.where(merge_data_unique["Kamar Tidur"] < lower_bound, lower_bound, merge_data_unique["Kamar Tidur"])
merge_data_unique["Kamar Tidur"] = np.where(merge_data_unique["Kamar Tidur"] > upper_bound, upper_bound, merge_data_unique["Kamar Tidur"])

NameError: name 'lower_bound' is not defined

In [98]:
index_outlier = []
def check_handle_outlier(column):
    global index_outlier
    print(column)
    # Calculate Q1 (25th percentile) and Q3 (75th percentile)
    Q1 = merge_data_unique[column].quantile(0.25)
    Q3 = merge_data_unique[column].quantile(0.75)

    # Calculate the IQR
    IQR = Q3 - Q1
    print(f"Nilai IQR untuk kolom {column} adalah {IQR}")
    # Define the lower and upper bounds for outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR    
    outliers = merge_data_unique[(merge_data_unique[column] < lower_bound) | (merge_data_unique[column] > upper_bound)]
    print(f"Jumlah baris yang terdeteksi outlier di kolom {column} adalah {outliers.shape[0]} baris\n")
    if len(outliers)!=0: 
        index_outlier = index_outlier + (outliers.index.values).tolist()
        # Handle outliers by capping and flooring
#         merge_data_unique[column] = np.where(merge_data_unique[column] < lower_bound, lower_bound, merge_data_unique[column])
#         merge_data_unique[column] = np.where(merge_data_unique[column] > upper_bound, upper_bound, merge_data_unique[column])

In [99]:
column_numeric = ["Kamar Tidur", "Kamar Mandi", "Luas Tanah", "Luas Bangunan", 
                  "Daya Listrik"]
for col in column_numeric:
    check_handle_outlier(col)

Kamar Tidur
Nilai IQR untuk kolom Kamar Tidur adalah 2.0
Jumlah baris yang terdeteksi outlier di kolom Kamar Tidur adalah 1165 baris

Kamar Mandi
Nilai IQR untuk kolom Kamar Mandi adalah 2.0
Jumlah baris yang terdeteksi outlier di kolom Kamar Mandi adalah 948 baris

Luas Tanah
Nilai IQR untuk kolom Luas Tanah adalah 237.0
Jumlah baris yang terdeteksi outlier di kolom Luas Tanah adalah 2485 baris

Luas Bangunan
Nilai IQR untuk kolom Luas Bangunan adalah 264.0
Jumlah baris yang terdeteksi outlier di kolom Luas Bangunan adalah 1981 baris

Daya Listrik
Nilai IQR untuk kolom Daya Listrik adalah 3283.247501529676
Jumlah baris yang terdeteksi outlier di kolom Daya Listrik adalah 2196 baris



In [100]:
merge_data_unique

,price,Kamar Tidur,Kamar Mandi,Luas Tanah,Luas Bangunan,Daya Listrik,Dapur,Ruang Makan,Ruang Tamu,Kondisi Perabotan,...,Pesanggrahan,Pulo Gadung,Sawah Besar,Senen,Setiabudi,Taman Sari,Tambora,Tanah Abang,Tanjung Priok,Tebet
0,2.200000e+09,3.0,2.0,84.0,79.0,2200.000000,1.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1.700000e+09,3.0,2.0,118.0,200.0,2200.000000,1.0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,2.470000e+09,2.0,2.0,76.0,90.0,2200.000000,1.0,1,1,0,...,0,1,0,0,0,0,0,0,0,0
3,8.280000e+08,2.0,2.0,31.0,56.0,2200.000000,1.0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4,1.320000e+09,3.0,2.0,48.0,67.0,2200.000000,1.0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27165,2.300000e+10,4.0,4.0,1000.0,750.0,5483.247502,1.0,1,1,0,...,0,0,0,0,0,0,0,0,1,0
27166,5.700000e+09,4.0,3.0,153.0,225.0,4400.000000,1.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27167,1.350000e+10,5.0,4.0,512.0,300.0,7700.000000,1.0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
27168,6.000000e+09,3.0,3.0,200.0,275.0,5483.247502,1.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [101]:
len(list(set(index_outlier)))

5254

In [102]:
print(f"Jumlah baris yang terdeteksi sebagai outlier adalah {len(list(set(index_outlier)))} baris")
merge_data_filtered = merge_data_unique.drop((list(set(index_outlier))))
merge_data_filtered

Jumlah baris yang terdeteksi sebagai outlier adalah 5254 baris


,price,Kamar Tidur,Kamar Mandi,Luas Tanah,Luas Bangunan,Daya Listrik,Dapur,Ruang Makan,Ruang Tamu,Kondisi Perabotan,...,Pesanggrahan,Pulo Gadung,Sawah Besar,Senen,Setiabudi,Taman Sari,Tambora,Tanah Abang,Tanjung Priok,Tebet
0,2.200000e+09,3.0,2.0,84.0,79.0,2200.000000,1.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1.700000e+09,3.0,2.0,118.0,200.0,2200.000000,1.0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,2.470000e+09,2.0,2.0,76.0,90.0,2200.000000,1.0,1,1,0,...,0,1,0,0,0,0,0,0,0,0
3,8.280000e+08,2.0,2.0,31.0,56.0,2200.000000,1.0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4,1.320000e+09,3.0,2.0,48.0,67.0,2200.000000,1.0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27162,6.700000e+09,6.0,3.0,200.0,300.0,3500.000000,1.0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
27166,5.700000e+09,4.0,3.0,153.0,225.0,4400.000000,1.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27167,1.350000e+10,5.0,4.0,512.0,300.0,7700.000000,1.0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
27168,6.000000e+09,3.0,3.0,200.0,275.0,5483.247502,1.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Feature Scalling

In [103]:
numeric_column = ["Kamar Tidur", "Kamar Mandi", "Luas Tanah", "Luas Bangunan", 
                  "Daya Listrik", "Dapur", "Jumlah Lantai", "Lebar Jalan", "Carport", "Kamar Pembantu"]
X = merge_data_filtered[numeric_column]  
y = merge_data_filtered['price']

In [104]:
merge_data_filtered

,price,Kamar Tidur,Kamar Mandi,Luas Tanah,Luas Bangunan,Daya Listrik,Dapur,Ruang Makan,Ruang Tamu,Kondisi Perabotan,...,Pesanggrahan,Pulo Gadung,Sawah Besar,Senen,Setiabudi,Taman Sari,Tambora,Tanah Abang,Tanjung Priok,Tebet
0,2.200000e+09,3.0,2.0,84.0,79.0,2200.000000,1.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1.700000e+09,3.0,2.0,118.0,200.0,2200.000000,1.0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,2.470000e+09,2.0,2.0,76.0,90.0,2200.000000,1.0,1,1,0,...,0,1,0,0,0,0,0,0,0,0
3,8.280000e+08,2.0,2.0,31.0,56.0,2200.000000,1.0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4,1.320000e+09,3.0,2.0,48.0,67.0,2200.000000,1.0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27162,6.700000e+09,6.0,3.0,200.0,300.0,3500.000000,1.0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
27166,5.700000e+09,4.0,3.0,153.0,225.0,4400.000000,1.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27167,1.350000e+10,5.0,4.0,512.0,300.0,7700.000000,1.0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
27168,6.000000e+09,3.0,3.0,200.0,275.0,5483.247502,1.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [105]:
minmax_scaler = MinMaxScaler()

X_minmax_scaler = minmax_scaler.fit_transform(X)

In [106]:
X_minmax_scaled = pd.DataFrame(X_minmax_scaler, columns=X.columns,index=X.index)
X_minmax_scaled = pd.concat([X_minmax_scaled, merge_data_filtered.drop(columns=numeric_column+['price'])], axis=1)
X_minmax_scaled

,Kamar Tidur,Kamar Mandi,Luas Tanah,Luas Bangunan,Daya Listrik,Dapur,Jumlah Lantai,Lebar Jalan,Carport,Kamar Pembantu,...,Pesanggrahan,Pulo Gadung,Sawah Besar,Senen,Setiabudi,Taman Sari,Tambora,Tanah Abang,Tanjung Priok,Tebet
0,0.285714,0.166667,0.120116,0.098237,0.183246,0.0,0.05,0.333333,0.0625,0.0,...,0,0,0,0,0,0,0,0,0,0
1,0.285714,0.166667,0.169320,0.250630,0.183246,0.0,0.05,0.333333,0.0000,0.0,...,0,0,0,0,0,0,0,0,0,0
2,0.142857,0.166667,0.108538,0.112091,0.183246,0.0,0.05,0.333333,0.0000,0.0,...,0,1,0,0,0,0,0,0,0,0
3,0.142857,0.166667,0.043415,0.069270,0.183246,0.0,0.05,0.333333,0.0000,0.0,...,0,0,0,0,0,0,0,0,0,0
4,0.285714,0.166667,0.068017,0.083123,0.183246,0.0,0.05,0.000000,0.0000,0.0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27162,0.714286,0.333333,0.287988,0.376574,0.319372,0.0,0.05,0.333333,0.0625,0.0,...,0,0,0,0,0,0,0,0,0,0
27166,0.428571,0.333333,0.219971,0.282116,0.413613,0.0,0.10,0.333333,0.0625,0.0,...,0,0,0,0,0,0,0,0,0,0
27167,0.571429,0.500000,0.739508,0.376574,0.759162,0.0,0.05,0.333333,0.0625,0.0,...,0,0,0,0,0,0,0,0,0,0
27168,0.285714,0.333333,0.287988,0.345088,0.527042,0.0,0.05,0.333333,0.0625,0.0,...,0,0,0,0,0,0,0,0,0,0


In [107]:
standard_scaler = StandardScaler()

X_standard_scaler = standard_scaler.fit_transform(X)

In [108]:
X_standard_scaler

array([[-0.64740923, -0.9761708 , -0.78258391, ..., -0.22618241,
         0.28081189, -0.22501708],
       [-0.64740923, -0.9761708 , -0.52688176, ..., -0.22618241,
        -1.15082255, -0.22501708],
       [-1.48306946, -0.9761708 , -0.84274912, ..., -0.22618241,
        -1.15082255, -0.22501708],
       ...,
       [ 1.02391122,  0.83229457,  2.43625493, ..., -0.22618241,
         0.28081189, -0.22501708],
       [-0.64740923, -0.07193812,  0.08981166, ..., -0.22618241,
         0.28081189, -0.22501708],
       [-0.64740923, -0.9761708 , -0.85026978, ..., -0.22618241,
        -1.15082255, -0.22501708]])

In [109]:
X_standard_scaled = pd.DataFrame(X_standard_scaler, columns=X.columns,index=X.index)
X_standard_scaled = pd.concat([X_standard_scaled,merge_data_filtered.drop(columns=numeric_column+['price'])], axis=1)
X_standard_scaled

,Kamar Tidur,Kamar Mandi,Luas Tanah,Luas Bangunan,Daya Listrik,Dapur,Jumlah Lantai,Lebar Jalan,Carport,Kamar Pembantu,...,Pesanggrahan,Pulo Gadung,Sawah Besar,Senen,Setiabudi,Taman Sari,Tambora,Tanah Abang,Tanjung Priok,Tebet
0,-0.647409,-0.976171,-0.782584,-1.034641,-1.101782,-0.238414,0.054155,-0.226182,0.280812,-0.225017,...,0,0,0,0,0,0,0,0,0,0
1,-0.647409,-0.976171,-0.526882,-0.203488,-1.101782,-0.238414,0.054155,-0.226182,-1.150823,-0.225017,...,0,0,0,0,0,0,0,0,0,0
2,-1.483069,-0.976171,-0.842749,-0.959082,-1.101782,-0.238414,0.054155,-0.226182,-1.150823,-0.225017,...,0,1,0,0,0,0,0,0,0,0
3,-1.483069,-0.976171,-1.181178,-1.192629,-1.101782,-0.238414,0.054155,-0.226182,-1.150823,-0.225017,...,0,0,0,0,0,0,0,0,0,0
4,-0.647409,-0.976171,-1.053327,-1.117070,-1.101782,-0.238414,0.054155,-2.445062,-1.150823,-0.225017,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27162,1.859571,-0.071938,0.089812,0.483416,-0.365521,-0.238414,0.054155,-0.226182,0.280812,-0.225017,...,0,0,0,0,0,0,0,0,0,0
27166,0.188251,-0.071938,-0.263659,-0.031762,0.144198,-0.238414,1.510995,-0.226182,0.280812,-0.225017,...,0,0,0,0,0,0,0,0,0,0
27167,1.023911,0.832295,2.436255,0.483416,2.013167,-0.238414,0.054155,-0.226182,0.280812,-0.225017,...,0,0,0,0,0,0,0,0,0,0
27168,-0.647409,-0.071938,0.089812,0.311690,0.757700,-0.238414,0.054155,-0.226182,0.280812,-0.225017,...,0,0,0,0,0,0,0,0,0,0


In [110]:
mean_daya_listrik = X_standard_scaled["Daya Listrik"].mean()
std_daya_listrik = X_standard_scaled["Daya Listrik"].std()

print("Mean of Daya Listrik:", mean_daya_listrik)
print("Standard Deviation of Daya Listrik:", std_daya_listrik)

Mean of Daya Listrik: -3.1735415759267527e-17
Standard Deviation of Daya Listrik: 1.0000232631175021


In [111]:
round(-3.1735415759267527e-17, 10)

-0.0

## Modelling

Splitting of MinMax Scaled data

In [117]:
max(y)

498000000000.0

In [112]:
# Split the data into training and testing sets (by default, 80% train, 20 test) 
X_train_minmax, X_test_minmax, y_train_minmax, y_test_minmax = train_test_split(X_minmax_scaled, y, test_size=0.2, 
                                                                                random_state=16)

In [113]:
X_train_minmax, X_test_minmax, y_train_minmax, y_test_minmax

(       Kamar Tidur  Kamar Mandi  Luas Tanah  Luas Bangunan  Daya Listrik  \
 4898      0.142857     0.333333    0.101302       0.096977      0.183246   
 19500     0.285714     0.333333    0.133140       0.401763      0.528796   
 7287      0.285714     0.166667    0.376266       0.250630      0.319372   
 16533     0.428571     0.333333    0.143271       0.285894      0.413613   
 9309      0.571429     0.500000    0.305355       0.259446      0.527042   
 ...            ...          ...         ...            ...           ...   
 22689     0.428571     0.333333    0.331404       0.288413      0.413613   
 614       1.000000     0.500000    0.432706       0.376574      0.527042   
 2280      0.428571     0.333333    0.287988       0.313602      0.298429   
 20272     0.571429     0.500000    0.128799       0.149874      0.527042   
 8299      0.142857     0.166667    0.108538       0.112091      0.183246   
 
           Dapur  Jumlah Lantai  Lebar Jalan  Carport  Kamar Pembantu  ...

In [114]:
# Split the data into training and testing sets (by default, 80% train, 20 test) 
X_train_standard, X_test_standard, y_train_standard, y_test_standard = train_test_split(X_standard_scaled, y, test_size=0.2, 
                                                                                random_state=16)

In [115]:
X_train_minmax

,Kamar Tidur,Kamar Mandi,Luas Tanah,Luas Bangunan,Daya Listrik,Dapur,Jumlah Lantai,Lebar Jalan,Carport,Kamar Pembantu,...,Pesanggrahan,Pulo Gadung,Sawah Besar,Senen,Setiabudi,Taman Sari,Tambora,Tanah Abang,Tanjung Priok,Tebet
4898,0.142857,0.333333,0.101302,0.096977,0.183246,0.000000,0.05,0.333333,0.0625,0.0,...,0,0,0,0,0,0,0,0,0,0
19500,0.285714,0.333333,0.133140,0.401763,0.528796,0.000000,0.20,0.333333,0.0625,0.0,...,0,0,0,0,0,0,0,0,0,0
7287,0.285714,0.166667,0.376266,0.250630,0.319372,0.000000,0.00,0.333333,0.0625,0.0,...,0,0,0,0,0,0,0,0,0,0
16533,0.428571,0.333333,0.143271,0.285894,0.413613,0.000000,0.10,0.333333,0.0000,0.0,...,0,0,0,0,0,0,0,0,0,0
9309,0.571429,0.500000,0.305355,0.259446,0.527042,0.000000,0.00,0.333333,0.0625,0.0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22689,0.428571,0.333333,0.331404,0.288413,0.413613,0.000000,0.05,0.666667,0.0625,0.0,...,0,0,0,0,0,0,0,0,0,0
614,1.000000,0.500000,0.432706,0.376574,0.527042,0.111111,0.05,0.333333,0.0625,0.0,...,0,0,0,0,0,0,0,0,0,0
2280,0.428571,0.333333,0.287988,0.313602,0.298429,0.000000,0.05,0.333333,0.0000,0.0,...,0,0,0,0,0,0,0,0,0,0
20272,0.571429,0.500000,0.128799,0.149874,0.527042,0.000000,0.00,0.333333,0.0625,0.0,...,0,0,0,0,0,0,0,0,0,0


Training

In [118]:
max(y)

498000000000.0

In [117]:
# Start the timer
start_time = time.time()
rf_regressor_minmax = RandomForestRegressor()
rf_regressor_minmax.fit(X_train_minmax, y_train_minmax)
# End the timer
end_time = time.time()
# Calculate the training time
training_time = end_time - start_time

print("Training time Random Forest Regression(MinMax):", training_time, "seconds")

Training time Random Forest Regression(MinMax): 12.896862745285034 seconds


In [118]:
rf_regressor_minmax.score(X_test_minmax, y_test_minmax)

0.8006773182744739

In [142]:
y_pred_minmax_rf = rf_regressor_minmax.predict(X_test_minmax)

In [165]:
# Evaluate the model
mse = mean_squared_error(y_test_minmax, y_pred_minmax_rf)
print("Root Mean Squared Error: ", np.sqrt(mse))

Root Mean Squared Error:  2817194101.45134


In [167]:
r2 = r2_score(y_test_minmax, y_pred_minmax_rf)
print("R-squared (R2) Score:", r2)

R-squared (R2) Score: 0.8006773182744739


In [120]:
# Start the timer
start_time = time.time()
rf_regressor_standard = RandomForestRegressor()
rf_regressor_standard.fit(X_train_standard, y_train_standard)
# End the timer
end_time = time.time()
# Calculate the training time
training_time = end_time - start_time

print("Training time Random Forest Regression(Standardization):", training_time, "seconds")

Training time Random Forest Regression(Standardization): 15.850452184677124 seconds


In [121]:
rf_regressor_standard.score(X_test_standard, y_test_standard)

0.8178056407658154

In [140]:
y_pred_standard_rf = rf_regressor_standard.predict(X_test_standard)

In [162]:
# Evaluate the model
mse = mean_squared_error(y_test_standard, y_pred_standard_rf)
print("Root Mean Squared Error: ", np.sqrt(mse))

Root Mean Squared Error:  2693431116.537486


In [161]:
r2 = r2_score(y_test_standard, y_pred_standard_rf)
print("R-squared (R2) Score:", r2)

R-squared (R2) Score: 0.8178056407658154


In [475]:
pd.DataFrame({'Actual': y_test_minmax, 'Predicted': y_pred_minmax_rf, 'error':y_test_minmax-y_pred_minmax_rf}).head(10)

,Actual,Predicted,error
14852,3.600000e+09,6.097400e+09,-2.497400e+09
1101,1.550000e+09,1.550000e+09,0.000000e+00
19458,4.390000e+09,4.564200e+09,-1.742000e+08
25389,5.500000e+09,5.348000e+09,1.520000e+08
26640,3.400000e+09,3.204300e+09,1.957000e+08
2992,3.900000e+09,4.010200e+09,-1.102000e+08
9555,2.050000e+09,1.901650e+09,1.483500e+08
9393,1.000000e+09,1.022167e+09,-2.216667e+07
20042,3.500000e+09,3.893200e+09,-3.932000e+08
18488,1.900000e+09,2.998100e+09,-1.098100e+09


XGBoost

In [135]:
xgb_regressor_minmax = XGBRegressor(n_estimators = 150)

In [136]:
# Start the timer
start_time = time.time()
# Train the model
xgb_regressor_minmax.fit(X_train_minmax, y_train_minmax)
# End the timer
end_time = time.time()
# Calculate the training time
training_time = end_time - start_time

print("Training time XGBoost Regression(MinMax):", training_time, "seconds")

Training time XGBoost Regression(MinMax): 1.2428743839263916 seconds


In [137]:
xgb_regressor_minmax.score(X_test_minmax, y_test_minmax)

0.8539923854197978

In [138]:
y_pred_minmax_xgb = xgb_regressor_minmax.predict(X_test_minmax)

In [157]:
# Evaluate the model
mse = mean_squared_error(y_test_minmax, y_pred_minmax_xgb)
print("Root Mean Squared Error: ", np.sqrt(mse))

Root Mean Squared Error:  2411160733.6625586


In [556]:
r2 = r2_score(y_test_minmax, y_pred_minmax_xgb)
print("R-squared (R2) Score:", r2)

R-squared (R2) Score: 0.8539923854197978


In [557]:
pd.DataFrame({'Actual': y_test_minmax, 'Predicted': y_pred_minmax_xgb, 'error':y_test_minmax-y_pred_minmax_xgb})

,Actual,Predicted,error
14852,3.600000e+09,5.366478e+09,-1.766478e+09
1101,1.550000e+09,1.692866e+09,-1.428664e+08
19458,4.390000e+09,4.942028e+09,-5.520283e+08
25389,5.500000e+09,5.034250e+09,4.657503e+08
26640,3.400000e+09,2.943025e+09,4.569751e+08
...,...,...,...
23015,5.800000e+09,5.545220e+09,2.547799e+08
2065,2.350000e+09,2.007352e+09,3.426478e+08
8382,2.000000e+09,2.995653e+09,-9.956531e+08
21037,5.200000e+09,4.156392e+09,1.043608e+09


In [149]:
xgb_regressor_standard = XGBRegressor(n_estimators = 150)

In [150]:
# Start the timer
start_time = time.time()
# Train the model
xgb_regressor_standard.fit(X_train_standard, y_train_standard)
# End the timer
end_time = time.time()
# Calculate the training time
training_time = end_time - start_time

print("Training time XGBoost Regression(Standardization):", training_time, "seconds")

Training time XGBoost Regression(Standardization): 1.2497239112854004 seconds


In [152]:
xgb_regressor_standard.score(X_test_standard, y_test_standard)

0.8515090937127757

In [566]:
y_pred_standard_xgb = xgb_regressor_standard.predict(X_test_standard)

In [567]:
y_pred_standard_xgb = np.abs(y_pred_standard_xgb)

In [568]:
# Evaluate the model
mse = mean_squared_error(y_test_standard, y_pred_standard_xgb)
print("Root Mean Squared Error: ", np.sqrt(mse))

Root Mean Squared Error:  2425241443.0203705


### Evaluasi

In [119]:
max(y)

498000000000.0

In [166]:
# Random Forest (Minmax)
y_pred_minmax_rf = rf_regressor_minmax.predict(X_test_minmax)
mse = mean_squared_error(y_test_minmax, y_pred_minmax_rf)
r2 = r2_score(y_test_minmax, y_pred_minmax_rf)
print("R-squared (R2) Score Random Forest Regression (MinMax):", r2)
print("Root Mean Squared Error Random Forest Regression (MinMax): ", np.sqrt(mse))
print()
# Random Forest (Standardization)
y_pred_standard_rf = rf_regressor_standard.predict(X_test_standard)
mse = mean_squared_error(y_test_standard, y_pred_standard_rf)
r2 = r2_score(y_test_standard, y_pred_standard_rf)
print("R-squared (R2) Score Random Forest Regression (Standardization):", r2)
print("Root Mean Squared Error Random Forest Regression (Standardization): ", np.sqrt(mse))
print()
# XGBoost (Minmax)
y_pred_minmax_xgb = xgb_regressor_minmax.predict(X_test_minmax)
mse = mean_squared_error(y_test_minmax, np.abs(y_pred_minmax_xgb))
r2 = r2_score(y_test_minmax, np.abs(y_pred_minmax_xgb))
print("R-squared (R2) Score XGBoost Regression (Minmax):", r2)
print("Root Mean Squared Error XGBoost Forest Regression (MinMax): ", np.sqrt(mse))
print()
# XGBoost (Standardization)
y_pred_standard_xgb = xgb_regressor_standard.predict(X_test_standard)
mse = mean_squared_error(y_test_standard, np.abs(y_pred_standard_xgb))
r2 = r2_score(y_test_standard, np.abs(y_pred_standard_xgb))
print("R-squared (R2) Score XGBoost Regression (Standardization):", r2)
print("Root Mean Squared Error XGBoost Regression (Standardization): ", np.sqrt(mse))

R-squared (R2) Score Random Forest Regression (MinMax): 0.8006773182744739
Root Mean Squared Error Random Forest Regression (MinMax):  2817194101.45134

R-squared (R2) Score Random Forest Regression (Standardization): 0.8178056407658154
Root Mean Squared Error Random Forest Regression (Standardization):  2693431116.537486

R-squared (R2) Score XGBoost Regression (Minmax): 0.8547469913270382
Root Mean Squared Error XGBoost Forest Regression (MinMax):  2404921904.5572042

R-squared (R2) Score XGBoost Regression (Standardization): 0.8522820939762207
Root Mean Squared Error XGBoost Regression (Standardization):  2425241443.0203705


In [151]:
r2 = r2_score(y_test_standard, y_pred_standard_xgb)
print("R-squared (R2) Score:", r2)

NameError: name 'y_pred_standard_xgb' is not defined

In [512]:
df = pd.DataFrame({'Actual': y_test_standard, 'Predicted': y_pred_standard_xgb, 'error':y_test_standard-y_pred_standard_xgb})
# df_filtered = df[df['Predicted'] <= 0]

In [513]:
df

,Actual,Predicted,error
14852,3.600000e+09,4.726349e+09,-1.126349e+09
1101,1.550000e+09,1.622726e+09,-7.272589e+07
19458,4.390000e+09,5.163074e+09,-7.730740e+08
25389,5.500000e+09,5.193665e+09,3.063350e+08
26640,3.400000e+09,3.113898e+09,2.861020e+08
...,...,...,...
23015,5.800000e+09,6.355748e+09,-5.557484e+08
2065,2.350000e+09,2.350215e+09,-2.146560e+05
8382,2.000000e+09,2.711890e+09,-7.118899e+08
21037,5.200000e+09,3.698407e+09,1.501593e+09


In [514]:
a_row = dict(merge_data_filtered.iloc[0,])

In [607]:
# Save scaler
with open('scaler_latest.pkl', 'wb') as f:
    pickle.dump(minmax_scaler, f)

# Save model
with open('model_latest.pkl', 'wb') as f:
    pickle.dump(xgb_regressor_minmax, f)

In [608]:
# Load the scaler
with open('scaler_latest.pkl', 'rb') as f:
    loaded_scaler = pickle.load(f)

# Load the model
with open('model_latest.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

In [151]:
merge_data_filtered.iloc[0,]

price                            2.200000e+09
Kamar Tidur                      3.000000e+00
Kamar Mandi                      2.000000e+00
Luas Tanah                       8.400000e+01
Luas Bangunan                    7.900000e+01
Daya Listrik                     2.200000e+03
Dapur                            1.000000e+00
Ruang Makan                      0.000000e+00
Ruang Tamu                       0.000000e+00
Kondisi Perabotan                0.000000e+00
Jumlah Lantai                    2.000000e+00
Terjangkau Internet              1.000000e+00
Lebar Jalan                      2.000000e+00
Hook                             0.000000e+00
Kondisi Properti                 2.000000e+00
Carport                          2.000000e+00
Kamar Pembantu                   1.000000e+00
HGB - Hak Guna Bangunan          0.000000e+00
HP - Hak Pakai                   0.000000e+00
HS - Hak Sewa                    0.000000e+00
Lainnya (PPJB,Girik,Adat,dll)    0.000000e+00
SHM - Sertifikat Hak Milik       1

In [126]:
del a_row['price']

In [133]:
a_row

{'Kamar Tidur': 3.0,
 'Kamar Mandi': 2.0,
 'Luas Tanah': 84.0,
 'Luas Bangunan': 79.0,
 'Daya Listrik': 2200.0,
 'Dapur': 1.0,
 'Ruang Makan': 0.0,
 'Ruang Tamu': 0.0,
 'Kondisi Perabotan': 0.0,
 'Jumlah Lantai': 2.0,
 'Terjangkau Internet': 1.0,
 'Lebar Jalan': 2.0,
 'Hook': 0.0,
 'Kondisi Properti': 2.0,
 'Carport': 2.0,
 'Kamar Pembantu': 1.0,
 'HGB - Hak Guna Bangunan': 0.0,
 'HP - Hak Pakai': 0.0,
 'HS - Hak Sewa': 0.0,
 'Lainnya (PPJB,Girik,Adat,dll)': 0.0,
 'SHM - Sertifikat Hak Milik': 1.0,
 'PAM atau PDAM': 1.0,
 'Sumur Bor': 0.0,
 'Sumur Galian': 0.0,
 'Sumur Pompa': 0.0,
 'Sumur Resapan': 0.0,
 'Jakarta Barat': 0.0,
 'Jakarta Pusat': 0.0,
 'Jakarta Selatan': 0.0,
 'Jakarta Timur': 1.0,
 'Jakarta Utara': 0.0}

In [171]:
a_row

{'Kamar Tidur': 3.0,
 'Kamar Mandi': 2.0,
 'Luas Tanah': 84.0,
 'Luas Bangunan': 79.0,
 'Daya Listrik': 2200.0,
 'Dapur': 1.0,
 'Ruang Makan': 0.0,
 'Ruang Tamu': 0.0,
 'Kondisi Perabotan': 0.0,
 'Jumlah Lantai': 2.0,
 'Terjangkau Internet': 1.0,
 'Lebar Jalan': 2.0,
 'Hook': 0.0,
 'Kondisi Properti': 2.0,
 'Carport': 2.0,
 'Kamar Pembantu': 1.0,
 'HGB - Hak Guna Bangunan': 0.0,
 'HP - Hak Pakai': 0.0,
 'HS - Hak Sewa': 0.0,
 'Lainnya (PPJB,Girik,Adat,dll)': 0.0,
 'SHM - Sertifikat Hak Milik': 1.0,
 'PAM atau PDAM': 1.0,
 'Sumur Bor': 0.0,
 'Sumur Galian': 0.0,
 'Sumur Pompa': 0.0,
 'Sumur Resapan': 0.0,
 'Jakarta Barat': 0.0,
 'Jakarta Pusat': 0.0,
 'Jakarta Selatan': 0.0,
 'Jakarta Timur': 1.0,
 'Jakarta Utara': 0.0}

In [154]:
row_df

,Kamar Tidur,Kamar Mandi,Luas Tanah,Luas Bangunan,Daya Listrik,Dapur,Ruang Makan,Ruang Tamu,Kondisi Perabotan,Jumlah Lantai,...,PAM atau PDAM,Sumur Bor,Sumur Galian,Sumur Pompa,Sumur Resapan,Jakarta Barat,Jakarta Pusat,Jakarta Selatan,Jakarta Timur,Jakarta Utara
0,0.285714,0.166667,0.120116,0.098237,0.183246,0.0,0.0,0.0,0.0,0.05,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [134]:
numeric_column = ["Kamar Tidur", "Kamar Mandi", "Luas Tanah", "Luas Bangunan", 
                  "Daya Listrik", "Dapur", "Jumlah Lantai", "Lebar Jalan", "Carport", "Kamar Pembantu"]

In [143]:
row_df = pd.DataFrame([a_row])

In [144]:
row_df[numeric_column] = minmax_scaler.transform(row_df[numeric_column])

In [146]:
row_df.columns

Index(['Kamar Tidur', 'Kamar Mandi', 'Luas Tanah', 'Luas Bangunan',
       'Daya Listrik', 'Dapur', 'Ruang Makan', 'Ruang Tamu',
       'Kondisi Perabotan', 'Jumlah Lantai', 'Terjangkau Internet',
       'Lebar Jalan', 'Hook', 'Kondisi Properti', 'Carport', 'Kamar Pembantu',
       'HGB - Hak Guna Bangunan', 'HP - Hak Pakai', 'HS - Hak Sewa',
       'Lainnya (PPJB,Girik,Adat,dll)', 'SHM - Sertifikat Hak Milik',
       'PAM atau PDAM', 'Sumur Bor', 'Sumur Galian', 'Sumur Pompa',
       'Sumur Resapan', 'Jakarta Barat', 'Jakarta Pusat', 'Jakarta Selatan',
       'Jakarta Timur', 'Jakarta Utara'],
      dtype='object')

In [147]:
print(xgb_regressor_minmax.predict(row_df))

[6.204868e+08]


In [148]:
from babel.numbers import format_currency

In [150]:
print(format_currency(xgb_regressor_minmax.predict(row_df)[0], 'IDR', locale='id_ID'))

Rp620.486.800,00


In [246]:
merge_data_filtered.iloc[0,]

price                            2.200000e+09
Kamar Tidur                      3.000000e+00
Kamar Mandi                      2.000000e+00
Luas Tanah                       8.400000e+01
Luas Bangunan                    7.900000e+01
Daya Listrik                     2.200000e+03
Dapur                            1.000000e+00
Ruang Makan                      0.000000e+00
Ruang Tamu                       0.000000e+00
Kondisi Perabotan                0.000000e+00
Jumlah Lantai                    2.000000e+00
Terjangkau Internet              1.000000e+00
Lebar Jalan                      2.000000e+00
Hook                             0.000000e+00
Kondisi Properti                 2.000000e+00
Carport                          2.000000e+00
Kamar Pembantu                   1.000000e+00
HGB - Hak Guna Bangunan          0.000000e+00
HP - Hak Pakai                   0.000000e+00
HS - Hak Sewa                    0.000000e+00
Lainnya (PPJB,Girik,Adat,dll)    0.000000e+00
SHM - Sertifikat Hak Milik       1

In [585]:
dictionary = {col: None for col in X_minmax_scaled.columns}

In [586]:
dictionary

{'Kamar Tidur': None,
 'Kamar Mandi': None,
 'Luas Tanah': None,
 'Luas Bangunan': None,
 'Daya Listrik': None,
 'Dapur': None,
 'Jumlah Lantai': None,
 'Lebar Jalan': None,
 'Carport': None,
 'Kamar Pembantu': None,
 'Ruang Makan': None,
 'Ruang Tamu': None,
 'Kondisi Perabotan': None,
 'Terjangkau Internet': None,
 'Hook': None,
 'Kondisi Properti': None,
 'HGB - Hak Guna Bangunan': None,
 'HP - Hak Pakai': None,
 'HS - Hak Sewa': None,
 'Lainnya (PPJB,Girik,Adat,dll)': None,
 'SHM - Sertifikat Hak Milik': None,
 'PAM atau PDAM': None,
 'Sumur Bor': None,
 'Sumur Galian': None,
 'Sumur Pompa': None,
 'Sumur Resapan': None,
 'Jakarta Barat': None,
 'Jakarta Pusat': None,
 'Jakarta Selatan': None,
 'Jakarta Timur': None,
 'Jakarta Utara': None,
 'Cakung': None,
 'Cempaka Putih': None,
 'Cengkareng': None,
 'Cilandak': None,
 'Cilincing': None,
 'Cipayung': None,
 'Ciracas': None,
 'Duren Sawit': None,
 'Gambir': None,
 'Grogol Petamburan': None,
 'Jagakarsa': None,
 'Jatinegara': None,

In [596]:
kamar_tidur = 3
dictionary["Kamar Tidur"] = kamar_tidur

kamar_mandi = 2
dictionary["Kamar Mandi"] = kamar_mandi

luas_bangunan = 100
dictionary["Luas Bangunan"] = luas_bangunan

luas_tanah = 114
dictionary["Luas Tanah"] = luas_tanah

daya_listrik =3200
dictionary["Daya Listrik"] = daya_listrik

dapur = 1
dictionary["Dapur"] = dapur

jumlah_lantai = 2
dictionary["Jumlah Lantai"] = jumlah_lantai

carport = 1
dictionary["Carport"] = carport

lebar_jalan = 2
dictionary["Lebar Jalan"]=2

kamar_pembantu = 0
dictionary["Kamar Pembantu"] = kamar_pembantu

ruang_makan = 1
dictionary["Ruang Makan"] = ruang_makan

ruang_tamu = 1
dictionary["Ruang Tamu"] = ruang_tamu

kondisi_perabotan = 2
dictionary["Kondisi Perabotan"] = kondisi_perabotan

terjangkau_internet = 1
dictionary["Terjangkau Internet"] = terjangkau_internet

hook = 0
dictionary["Hook"] = hook

kondisi_properti = 3
dictionary["Kondisi Properti"] = kondisi_properti

sertifikat = "SHM - Sertifikat Hak Milik"
    
sertifikat_dict = {
    "SHM - Sertifikat Hak Milik": 0,
    'HGB - Hak Guna Bangunan': 0,
    'HP - Hak Pakai': 0,
    'HS - Hak Sewa': 0,
    'Lainnya (PPJB,Girik,Adat,dll)': 0
}

# Update the dictionary values
for key in sertifikat_dict:
    dictionary[key] = 1 if key == sertifikat else 0

water_source = "PAM atau PDAM"
water_source_dict = {
    'PAM atau PDAM': 0,
    'Sumur Bor': 0,
    'Sumur Galian': 0,
    'Sumur Pompa': 0,
    'Sumur Resapan': 0
}

# Update the dictionary values
for key in water_source_dict:
    dictionary[key] = 1 if key == water_source else 0
    

# The region you want to set
region = "Jakarta Timur"
region_dict = {
    'Jakarta Barat': 0,
    'Jakarta Pusat': 0,
    'Jakarta Selatan': 0,
    'Jakarta Timur': 0,
    'Jakarta Utara': 0
}
# Set the value of the specific region to 1, others remain 0
for key in region_dict:
    dictionary[key] = 1 if key == region else 0
    

kecamatan = "Cakung"
kecamatan_list = [
    'Cakung', 'Cempaka Putih', 'Cengkareng', 'Cilandak', 'Cilincing', 'Cipayung', 
    'Ciracas', 'Duren Sawit', 'Gambir', 'Grogol Petamburan', 'Jagakarsa', 
    'Jatinegara', 'Johar Baru', 'Kalideres', 'Kebayoran Baru', 'Kebayoran Lama',
    'Kebon Jeruk', 'Kelapa Gading', 'Kemayoran', 'Kembangan', 'Koja',
    'Kramat Jati', 'Makasar', 'Mampang Prapatan', 'Matraman', 'Menteng',
    'Pademangan', 'Palmerah', 'Pancoran', 'Pasar Minggu', 'Pasar Rebo',
    'Penjaringan', 'Pesanggrahan', 'Pulo Gadung', 'Sawah Besar', 'Senen',
    'Setiabudi', 'Taman Sari', 'Tambora', 'Tanah Abang', 'Tanjung Priok',
    'Tebet'
]

for key in kecamatan_list:
    dictionary[key] = 1 if key == kecamatan else 0


In [591]:
for key in region_dict:
    dictionary[key] = 1 if key == water_source else 0

In [597]:
dictionary

{'Kamar Tidur': 3,
 'Kamar Mandi': 2,
 'Luas Tanah': 114,
 'Luas Bangunan': 100,
 'Daya Listrik': 3200,
 'Dapur': 1,
 'Jumlah Lantai': 2,
 'Lebar Jalan': 2,
 'Carport': 1,
 'Kamar Pembantu': 0,
 'Ruang Makan': 1,
 'Ruang Tamu': 1,
 'Kondisi Perabotan': 2,
 'Terjangkau Internet': 1,
 'Hook': 0,
 'Kondisi Properti': 3,
 'HGB - Hak Guna Bangunan': 0,
 'HP - Hak Pakai': 0,
 'HS - Hak Sewa': 0,
 'Lainnya (PPJB,Girik,Adat,dll)': 0,
 'SHM - Sertifikat Hak Milik': 1,
 'PAM atau PDAM': 1,
 'Sumur Bor': 0,
 'Sumur Galian': 0,
 'Sumur Pompa': 0,
 'Sumur Resapan': 0,
 'Jakarta Barat': 0,
 'Jakarta Pusat': 0,
 'Jakarta Selatan': 0,
 'Jakarta Timur': 1,
 'Jakarta Utara': 0,
 'Cakung': 1,
 'Cempaka Putih': 0,
 'Cengkareng': 0,
 'Cilandak': 0,
 'Cilincing': 0,
 'Cipayung': 0,
 'Ciracas': 0,
 'Duren Sawit': 0,
 'Gambir': 0,
 'Grogol Petamburan': 0,
 'Jagakarsa': 0,
 'Jatinegara': 0,
 'Johar Baru': 0,
 'Kalideres': 0,
 'Kebayoran Baru': 0,
 'Kebayoran Lama': 0,
 'Kebon Jeruk': 0,
 'Kelapa Gading': 0,
 '

In [598]:
numeric_column = ["Kamar Tidur", "Kamar Mandi", "Luas Tanah", "Luas Bangunan", 
                  "Daya Listrik", "Dapur", "Jumlah Lantai", "Lebar Jalan", "Carport", "Kamar Pembantu"]

In [610]:
row_df = pd.DataFrame([dictionary])

In [611]:
row_df[numeric_column]

,Kamar Tidur,Kamar Mandi,Luas Tanah,Luas Bangunan,Daya Listrik,Dapur,Jumlah Lantai,Lebar Jalan,Carport,Kamar Pembantu
0,3,2,114,100,3200,1,2,2,1,0


In [612]:
row_df[numeric_column] = loaded_scaler.transform(row_df[numeric_column])

In [613]:
print(loaded_model.predict(row_df))

[2.2210565e+09]


In [614]:
from babel.numbers import format_currency
print(format_currency(xgb_regressor_minmax.predict(row_df)[0], 'IDR', locale='id_ID'))

Rp2.221.056.500,00


In [428]:
score = []
for i in range(0,100):
    X_train_standard, X_test_standard, y_train_standard, y_test_standard = train_test_split(X_standard_scaled, y, test_size=0.2, 
                                                                                random_state=i)
    xgb_regressor_standard = XGBRegressor(objective='reg:squarederror')
    xgb_regressor_standard.fit(X_train_standard, y_train_standard)
    y_pred_standard_xgb = xgb_regressor_standard.predict(X_test_standard)
    score.append(xgb_regressor_standard.score(X_test_standard, y_test_standard))
    
    

In [462]:
np.argmax(score)

16

In [533]:
max(score)

0.8490250964575325

In [430]:
np.argmin(score)

42

In [122]:

# Define the parameter grid
param_grid = {
    'max_depth': [3, 6, 9],
    'learning_rate': [0.01, 0.1, 0.3],
    'n_estimators': [100, 200, 300],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'gamma': [0, 0.1, 0.2],
    'reg_alpha': [0, 0.1, 0.2],
    'reg_lambda': [0, 0.1, 0.2]
}

# Create XGBoost Regressor model
xgb_regressor = XGBRegressor(objective='reg:squarederror', random_state=42)

# Instantiate the GridSearchCV object
grid_search = GridSearchCV(estimator=xgb_regressor, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')

# Perform grid search
grid_search.fit(X_train_minmax, y_train_minmax)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Score (Negative MSE):", best_score)

# Predict on the testing set using the best model
y_pred = grid_search.best_estimator_.predict(X_test_minmax)


Fitting 3 folds for each of 6561 candidates, totalling 19683 fits


KeyboardInterrupt: 

In [115]:
# Define hyperparameters
max_depth = 20
min_samples_leaf = 1
min_samples_split = 2
n_estimators = 200

# Create RandomForestRegressor with specified hyperparameters
rf_regressor_standard = RandomForestRegressor(
    max_depth=max_depth,
    min_samples_leaf=min_samples_leaf,
    min_samples_split=min_samples_split,
    n_estimators=n_estimators
)
rf_regressor_standard.fit(X_train_standard, y_train_standard)

RandomForestRegressor(max_depth=20, n_estimators=200)

In [122]:
y_pred_standard = rf_regressor_standard.predict(X_test_standard)

In [126]:
# Evaluate the model
mse = mean_squared_error(y_test_standard, y_pred_standard)
print("Mean Squared Error:", mse)

Mean Squared Error: 9.85924822209058e+22


In [127]:
r2 = r2_score(y_test_standard, y_pred_standard)
print("R-squared (R2) Score:", r2)

R-squared (R2) Score: 0.00769943352379987
